____
# Secretaría de Paz y Cultura Ciudadana - SPCC
## Oservatorio de Paz y Convivencia - OPC
### PROCEDIMIENTO PARA CURACION DE DATOS BD PONAL
#### Mauricio Sedano - Lider Observatorio
#### Andrés Suárez - Procedimientos
#### Actualización 17ABR2019

____

## TRATAMIENTO DE LIMPIEZA

En esta sección se hará el tratamiento de los datos de todas las DB de los delitos a nivel nacional. 
Los datos a cargar debera  ser  dataframe merge "df_delito_2010a2019_raw_fecha.csv

Las operaciones a realizar son:

### ETAPA 1. FORMATOS y DERIVADOS
1. Cargar las DB de los 6 delitos ('AMENAZA','DELITO SEXUAL','HOMICIDIO','HURTO PERSONA', 'LESIONES PERSONALES', 'VIOLENCIA INTRAFAMILIAR')
2. Adicionar a cada DB la columna ["CONFLICTIVIDAD"], que categoriza a cada delito
3. MERGE todos las seis (6) DB para generar una (1) DB compilada. Para aplicar tratamiento como un todo. 
4. Verificacion de datos NULL y NaN (Sacar estadisticas)
5. Ajuste del formato de la columna FECHA y  derivar columnas ['FECHA_FORMATO'], ['AÑO'], ['MES'], ['DIA_NUMERO'], ['SEMANA_NUMERO'], ['DIA_JULIANO'], ['FECHA_ESPECIAL']
6. Ajuste del formato de la columna HORA y  derivar columnas ['HORA_FORMATO_24h'],['HORA_RANGO_24x']
7. Ajuste de la columna ['SEXO'] en la misma columna. Se mapea con el diccionario 
{'MASCULINO':'HOMBRE', 'FEMENINO':'MUJER', '-':'SIN DATO', 'NO REPORTADO':'SIN DATO', 'NO REPORTA':'SIN DATO'}
8. Ajuste del formato a númerico de la columna ['EDAD'] y derivar columnas ['EDAD_FORMATO'],['EDAD_5Q_80mas'],['EDAD_NNAJ']. Se usan los siguientes criterior para depurar la columna ['EDAD_FORMATO'], a) ['EDAD_FORMATO'] < 0 = np.nan b) ['EDAD_FORMATO'] > 115 = np.nan
9. Ajueste campo ['BARRIO'] y derivar columnas ['BARRIO_TEMP'], ['ESTACION_COD_DEV'], [COMUNA_DEV]

### ETAPA 2. TRATAMIENTO PARA EL CASO ['CIUDAD'] CALI(CT)

1. Seleccionar y crear dataframe para datos de sólo CALI (CT) con los 6 delitos ('AMENAZA','DELITO SEXUAL','HOMICIDIO','HURTO PERSONA', 'LESIONES PERSONALES', 'VIOLENCIA INTRAFAMILIAR')
2. Limpiar la columna ['BARRIO_NOMBRE'] aplicando diccionario
3. Mapeo columna ['BARRIO_OFICIAL'] con bases de datos oficial y derivar ['BARRIO_COD'], ['COMUNA'],['AREA'] 

____

### **ETAPA 1. FORMATOS y DERIVADOS**


#### **1.1 CARGA DE ARCHIVOS**

In [1]:
def listar_csv(ruta):
    import os
    import glob

    files = [f for f in glob.glob(ruta + "**/*.csv", recursive=False)]
    names = [os.path.basename(x) for x in glob.glob(ruta + "**/*.csv")]
    return files, names

def print_table(data, cols, wide):
    '''Prints formatted data on columns of given width.'''
    n, r = divmod(len(data), cols)
    pat = '{{:{}}}'.format(wide)
    line = '\n'.join(pat * cols for _ in range(n))
    last_line = pat * r
    print(line.format(*data))
    print(last_line.format(*data[n*cols:]))

In [2]:
import getpass
import os
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox

root = tk.Tk()
#root.withdraw()
b_sourceDIR = tk.Button(root, text = "OK", width = 20, height = 3, command= root.destroy).pack()
input_dir = filedialog.askdirectory(parent=root, initialdir= r'C:\Users\{}\Desktop'.format(getpass.getuser()), title = "Select Input folder")
#output_dir = filedialog.askdirectory(parent=root, initialdir= r'C:\Users\{}\Desktop'.format(getpass.getuser()), title = "Select Output folder")
root.mainloop()

In [3]:
lista_csv, lista_names = listar_csv(input_dir)
#print_table(lista_names, 1, 1)

In [4]:
import pandas as pd

nombre_base_amz = lista_csv[0]
nombre_base_ds = lista_csv[1]
nombre_base_ho = lista_csv[2]
nombre_base_hp = lista_csv[3]
nombre_base_lp = lista_csv[4]
nombre_base_vi = lista_csv[5]

df_2010u2018_amz_raw = pd.read_csv(nombre_base_amz)
df_2010u2018_ds_raw = pd.read_csv(nombre_base_ds)
df_2010u2018_ho_raw = pd.read_csv(nombre_base_ho)
df_2010u2018_hp_raw = pd.read_csv(nombre_base_hp)
df_2010u2018_lp_raw = pd.read_csv(nombre_base_lp)
df_2010u2018_vi_raw = pd.read_csv(nombre_base_vi)

print(df_2010u2018_amz_raw.shape)
print(df_2010u2018_ds_raw.shape)
print(df_2010u2018_ho_raw.shape)
print(df_2010u2018_hp_raw.shape)
print(df_2010u2018_lp_raw.shape)
print(df_2010u2018_vi_raw.shape)

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (11,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(249248, 20)
(152556, 21)
(125611, 20)
(1145265, 20)
(848939, 20)
(530915, 20)


In [5]:
print(df_2010u2018_amz_raw.columns.values)
print(df_2010u2018_ds_raw.columns.values)
print(df_2010u2018_ho_raw.columns.values)
print(df_2010u2018_hp_raw.columns.values)
print(df_2010u2018_lp_raw.columns.values)
print(df_2010u2018_vi_raw.columns.values)

['FECHA' 'DEPARTAMENTO_' 'MUNICIPIO' 'DIA' 'HORA' 'BARRIO' 'ZONA'
 'CLASE_SITIO' 'ARMA_EMPLEADA' 'MOVIL_AGRESOR' 'MOVIL_VICTIMA' 'EDAD'
 'SEXO' 'ESTADO_CIVIL' 'PAIS_NACIMIENTO' 'CLASE_EMPLEADO' 'PROFESION'
 'ESCOLARIDAD' 'CODIGO_DANE' 'CANTIDAD']
['FECHA' 'DEPARTAMENTO_' 'MUNICIPIO' 'DIA' 'HORA' 'BARRIO' 'ZONA'
 'CLASE_SITIO' 'ARMA_EMPLEADA' 'MOVIL_AGRESOR' 'MOVIL_VICTIMA' 'EDAD'
 'SEXO' 'ESTADO_CIVIL' 'PAIS_NACIMIENTO' 'CLASE_EMPLEADO' 'PROFESION'
 'ESCOLARIDAD' 'CODIGO_DANE' 'CANTIDAD' 'DELITO']
['FECHA' 'DEPARTAMENTO_' 'MUNICIPIO' 'DIA' 'HORA' 'BARRIO' 'ZONA'
 'CLASE_SITIO' 'ARMA_EMPLEADA' 'MOVIL_AGRESOR' 'MOVIL_VICTIMA' 'EDAD'
 'SEXO' 'ESTADO_CIVIL' 'PAIS_NACIMIENTO' 'CLASE_EMPLEADO' 'PROFESION'
 'ESCOLARIDAD' 'CODIGO_DANE' 'CANTIDAD']
['FECHA' 'DEPARTAMENTO_' 'MUNICIPIO' 'DIA' 'HORA' 'BARRIO' 'ZONA'
 'CLASE_SITIO' 'ARMA_EMPLEADA' 'MOVIL_AGRESOR' 'MOVIL_VICTIMA' 'EDAD'
 'SEXO' 'ESTADO_CIVIL' 'PAIS_NACIMIENTO' 'CLASE_EMPLEADO' 'PROFESION'
 'ESCOLARIDAD' 'CODIGO_DANE' 'CANTIDAD']
['F

#### **1.2 ADICIONAR COLUMNA ['CONFLICTIVIDAD']**

> **NOTA**
Se adiciona a la DB la columna ["CONFLICTIVIDAD"]. Se clasifica la conflictividad con las siguientes categorias ('AMENAZA','DELITO SEXUAL','HOMICIDIO','HURTO PERSONA', 'LESIONES PERSONALES', 'VIOLENCIA INTRAFAMILIAR')

In [6]:
df_2010u2018_amz_raw['CONFLICTIVIDAD'] = 'AMENAZA'
df_2010u2018_ds_raw['CONFLICTIVIDAD'] = 'DELITO SEXUAL'
df_2010u2018_ho_raw['CONFLICTIVIDAD'] = 'HOMICIDIO'
df_2010u2018_hp_raw['CONFLICTIVIDAD'] = 'HURTO PERSONA'
df_2010u2018_lp_raw['CONFLICTIVIDAD'] = 'LESIONES PERSONALES'
df_2010u2018_vi_raw['CONFLICTIVIDAD'] = 'VIOLENCIA INTRAFAMILIAR'

#### **1.3 MERGE DBs ** 

> **NOTA**
Se hace el merge para posterirormente hacer el proceso sólo como un hilo.

In [7]:
dataframes = [df_2010u2018_amz_raw, 
              df_2010u2018_ds_raw,
              df_2010u2018_ho_raw, 
              df_2010u2018_hp_raw, 
              df_2010u2018_lp_raw,
              df_2010u2018_vi_raw]

df_gen_2010a2018 = pd.concat(dataframes, ignore_index=True, sort=False)

print('El tamaño del Dataframe es:', df_gen_2010a2018.shape)

El tamaño del Dataframe es: (3052534, 22)


In [8]:
df_gen_2010a2018.sample(5)

,FECHA,DEPARTAMENTO_,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE_SITIO,ARMA_EMPLEADA,MOVIL_AGRESOR,...,SEXO,ESTADO_CIVIL,PAIS_NACIMIENTO,CLASE_EMPLEADO,PROFESION,ESCOLARIDAD,CODIGO_DANE,CANTIDAD,CONFLICTIVIDAD,DELITO
1720189,2010-11-20 00:00:00,CUNDINAMARCA,COTA,SÁBADO,18:00:00,VER.CENTRO,URBANA,VIAS PUBLICAS,CONTUNDENTES,A PIE,...,MASCULINO,SOLTERO,COLOMBIA,EMPLEADO PARTICULAR,NOREPORTADO,SECUNDARIA,25214000,1.0,LESIONES PERSONALES,NaN
1411872,02/15/2018 12:00:00 AM,ANTIOQUIA,MEDELLÍN (CT),JUEVES,12/31/1899 02:20:00 PM,CUARTA BRIGADA C-11,URBANA,RODANDO SOBRE VIA - VIA PUBLICA,ARMA DE FUEGO,PASAJERO MOTOCICLETA,...,MASCULINO,CASADO,COLOMBIA,EMPLEADO PARTICULAR,-,SUPERIOR,"5,001,000",1.0,HURTO PERSONA,NaN
1509610,2018-07-07 00:00:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),SÁBADO,12/31/1899 09:00:00 AM,QUINTA PAREDES E-13,URBANA,VIAS PUBLICAS,ARMA BLANCA / CORTOPUNZANTE,A PIE,...,MASCULINO,UNION LIBRE,COLOMBIA,EMPLEADO PARTICULAR,-,SUPERIOR,"11,001,000",1.0,HURTO PERSONA,NaN
97315,2015-06-10,CUNDINAMARCA,CABRERA,MIÉRCOLES,00:00:00,LA CULEBRERA,URBANA,VIAS PUBLICAS,SIN EMPLEO DE ARMAS,A PIE,...,MASCULINO,SOLTERO,-,EMPLEADO PARTICULAR,-,PRIMARIA,25120000,1.0,AMENAZA,NaN
738643,2012-01-19 00:00:00,PUTUMAYO,MOCOA (CT),JUEVES,11:30:00,AV. SAN FRANCISCO,URBANA,HOSPITALES,SIN EMPLEO DE ARMAS,A PIE,...,FEMENINO,SOLTERO,COLOMBIA,EMPLEADO PARTICULAR,NO REPORTADA,SUPERIOR,86001000,1.0,HURTO PERSONA,NaN


#### **1.4 VERIFICACIÓN DE DATOS NULL Y NAN ** 
> **Nota**
Se verificará si una de las columnas de los dataframes contiene datos nulos o Valore NaN. Columnas que indique False significa que no tiene datos nulos en caso que sea True es porque si.  

In [9]:
print("Resumen de la información básica sobre el Dataframe y sus datos:")
print(df_gen_2010a2018.info())

Resumen de la información básica sobre el Dataframe y sus datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3052534 entries, 0 to 3052533
Data columns (total 22 columns):
FECHA              object
DEPARTAMENTO_      object
MUNICIPIO          object
DIA                object
HORA               object
BARRIO             object
ZONA               object
CLASE_SITIO        object
ARMA_EMPLEADA      object
MOVIL_AGRESOR      object
MOVIL_VICTIMA      object
EDAD               object
SEXO               object
ESTADO_CIVIL       object
PAIS_NACIMIENTO    object
CLASE_EMPLEADO     object
PROFESION          object
ESCOLARIDAD        object
CODIGO_DANE        object
CANTIDAD           float64
CONFLICTIVIDAD     object
DELITO             object
dtypes: float64(1), object(21)
memory usage: 512.4+ MB
None


In [10]:
df_resumen =pd.DataFrame(df_gen_2010a2018.dtypes, columns=['Type'])
df_resumen['is_null'] =df_gen_2010a2018.isnull().any()
df_resumen['is_null_cant'] = df_gen_2010a2018.isnull().sum()
df_resumen['is_null_porc'] = df_resumen['is_null_cant']*100/ len(df_gen_2010a2018.index)
df_resumen['is_nan']=df_gen_2010a2018.isna().any()
df_resumen['is_nan_cant']=df_gen_2010a2018.isna().sum()

In [11]:
df_resumen

,Type,is_null,is_null_cant,is_null_porc,is_nan,is_nan_cant
FECHA,object,True,4774,0.156395,True,4774
DEPARTAMENTO_,object,False,0,0.000000,False,0
MUNICIPIO,object,False,0,0.000000,False,0
DIA,object,False,0,0.000000,False,0
HORA,object,False,0,0.000000,False,0
BARRIO,object,True,15799,0.517570,True,15799
ZONA,object,False,0,0.000000,False,0
CLASE_SITIO,object,False,0,0.000000,False,0
ARMA_EMPLEADA,object,False,0,0.000000,False,0
MOVIL_AGRESOR,object,False,0,0.000000,False,0


#### **1.5 AJUSTE DEL FORMATO DE LA COLUMNA ['FECHA'] y sus derivadas.** 
> **Nota**
Se derivan las columnas ['FECHA_FORMATO'], ['AÑO'], ['MES'], ['DIA_NUMERO'], ['SEMANA_NUMERO'], ['DIA_JULIANO'], ['FECHA_ESPECIAL'].

### 2.1.1 Ajuste de Formato de fecha.

In [12]:
#columna fecha se pasa a nueva columna con formato fecha. 
#Se derivan las demas enunciadas

df_gen_2010a2018['FECHA_FORMATO'] = pd.to_datetime(df_gen_2010a2018['FECHA'])  

df_gen_2010a2018['AÑO'] = df_gen_2010a2018['FECHA_FORMATO'].dt.year
df_gen_2010a2018['MES']  = df_gen_2010a2018['FECHA_FORMATO'].dt.month
df_gen_2010a2018['DIA_NUMERO']    = df_gen_2010a2018['FECHA_FORMATO'].dt.day
df_gen_2010a2018['SEMANA_NUMERO'] = df_gen_2010a2018['FECHA_FORMATO'].dt.week
df_gen_2010a2018['DIA_JULIANO']   = df_gen_2010a2018['FECHA_FORMATO'].dt.dayofyear


In [13]:
print('años', sorted(set(df_gen_2010a2018.AÑO.unique())))
print('meses',sorted(set(df_gen_2010a2018.MES.unique())))
print('dias en numero', sorted(set(df_gen_2010a2018.DIA_NUMERO.unique())))
print('semana', sorted(set(df_gen_2010a2018.SEMANA_NUMERO.unique())))
#print('dias julianos', df_gen_2010a2018.dia_juliano.unique())

años [2016.0, 2017.0, 2018.0, 2019.0, nan, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2015.0]
meses [nan, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0]
dias en numero [nan, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0]
semana [nan, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0]


In [14]:
print("Resumen de validación de conversión del formato FECHA:")

print('Son iguales los datos de la columnas FECHA y AÑO para el año 2010= ', 
      (df_gen_2010a2018['AÑO']==2010).sum() == df_gen_2010a2018['FECHA'].str.contains('2010').sum())

print('Son iguales los datos de la columnas FECHA y AÑO para el año 2011= ',
      (df_gen_2010a2018['AÑO']==2011).sum() == df_gen_2010a2018['FECHA'].str.contains('2011').sum())

print('Son iguales los datos de la columnas FECHA y AÑO para el año 2012= ',
      (df_gen_2010a2018['AÑO']==2012).sum() == df_gen_2010a2018['FECHA'].str.contains('2012').sum())

print('Son iguales los datos de la columnas FECHA y AÑO para el año 2013= ',
      (df_gen_2010a2018['AÑO']==2013).sum() == df_gen_2010a2018['FECHA'].str.contains('2013').sum())

print('Son iguales los datos de la columnas FECHA y AÑO para el año 2014= ',
      (df_gen_2010a2018['AÑO']==2014).sum() == df_gen_2010a2018['FECHA'].str.contains('2014').sum())

print('Son iguales los datos de la columnas FECHA y AÑO para el año 2015= ',
      (df_gen_2010a2018['AÑO']==2015).sum() == df_gen_2010a2018['FECHA'].str.contains('2015').sum())

print('Son iguales los datos de la columnas FECHA y AÑO para el año 2016= ',
      (df_gen_2010a2018['AÑO']==2016).sum() == df_gen_2010a2018['FECHA'].str.contains('2016').sum())

print('Son iguales los datos de la columnas FECHA y AÑO para el año 2017= ',
      (df_gen_2010a2018['AÑO']==2017).sum() == df_gen_2010a2018['FECHA'].str.contains('2017').sum())

print('Son iguales los datos de la columnas FECHA y AÑO para el año 2018= ',
      (df_gen_2010a2018['AÑO']==2018).sum() == df_gen_2010a2018['FECHA'].str.contains('2018').sum())

print('Son iguales los datos de la columnas FECHA y AÑO para el año 2019= ',
      (df_gen_2010a2018['AÑO']==2019).sum() == df_gen_2010a2018['FECHA'].str.contains('2019').sum())

from IPython.display import display
display(df_gen_2010a2018.sample(5))

Resumen de validación de conversión del formato FECHA:
Son iguales los datos de la columnas FECHA y AÑO para el año 2010=  True
Son iguales los datos de la columnas FECHA y AÑO para el año 2011=  True
Son iguales los datos de la columnas FECHA y AÑO para el año 2012=  True
Son iguales los datos de la columnas FECHA y AÑO para el año 2013=  True
Son iguales los datos de la columnas FECHA y AÑO para el año 2014=  True
Son iguales los datos de la columnas FECHA y AÑO para el año 2015=  True
Son iguales los datos de la columnas FECHA y AÑO para el año 2016=  True
Son iguales los datos de la columnas FECHA y AÑO para el año 2017=  True
Son iguales los datos de la columnas FECHA y AÑO para el año 2018=  True
Son iguales los datos de la columnas FECHA y AÑO para el año 2019=  True


,FECHA,DEPARTAMENTO_,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE_SITIO,ARMA_EMPLEADA,MOVIL_AGRESOR,...,CODIGO_DANE,CANTIDAD,CONFLICTIVIDAD,DELITO,FECHA_FORMATO,AÑO,MES,DIA_NUMERO,SEMANA_NUMERO,DIA_JULIANO
466359,2014-03-23,ANTIOQUIA,GIRARDOTA,DOMINGO,05:00:00,VDA. LA MESETA,RURAL,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,...,5308000,1.0,HOMICIDIO,NaN,2014-03-23,2014.0,3.0,23.0,12.0,82.0
1900614,2013-08-04 00:00:00,CESAR,AGUSTÍN CODAZZI,DOMINGO,17:40:00,CENTRO,URBANA,VIAS PUBLICAS,ARMA BLANCA / CORTOPUNZANTE,A PIE,...,20013000,1.0,LESIONES PERSONALES,NaN,2013-08-04,2013.0,8.0,4.0,31.0,216.0
2284060,2017-10-21 00:00:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),SÁBADO,13:10:00,DINDALITO E-8,URBANA,CASAS DE HABITACION,CONTUNDENTES,A PIE,...,11001000,1.0,LESIONES PERSONALES,NaN,2017-10-21,2017.0,10.0,21.0,42.0,294.0
2865195,2017-02-16 00:00:00,CAQUETÁ,FLORENCIA (CT),JUEVES,01:30:00,EL RAICERO,URBANA,CASAS DE HABITACION,CONTUNDENTES,A PIE,...,18001000,1.0,VIOLENCIA INTRAFAMILIAR,NaN,2017-02-16,2017.0,2.0,16.0,7.0,47.0
3002476,2018-07-15 00:00:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),DOMINGO,17:00:00,BELLA SUIZA E-1,URBANA,VIAS PUBLICAS,NO REPORTADO,A PIE,...,11001000,1.0,VIOLENCIA INTRAFAMILIAR,NaN,2018-07-15,2018.0,7.0,15.0,28.0,196.0


In [15]:
def fechas_especiales(fecha):
    
    import math
    
    fecha_ = str(fecha.month).zfill(2)+'-'+str(fecha.day).zfill(2)
   
    switcher = {
        '01-01': 'AÑO NUEVO',
        '10-31': 'HALLOWEN',
        '12-07': 'DIA VELITAS',
        '12-24': 'NAVIDAD',
        '12-25': 'FERIA CALI',
        '12-26': 'FERIA CALI',
        '12-27': 'FERIA CALI',
        '12-28': 'FERIA CALI',
        '12-29': 'FERIA CALI',
        '12-30': 'FERIA CALI',
        '12-31': 'FIN DE AÑO'}
    
    fecha_especial = (switcher.get(fecha_))
    
    #if fecha_especial==None:
     #   week_in_month = math.ceil(fecha.day/7)
     #   fecha_especial = week_in_month
        
    return fecha_especial



In [16]:
df_gen_2010a2018['FECHAS_ESPECIALES'] = df_gen_2010a2018.FECHA_FORMATO.apply(fechas_especiales)
df_gen_2010a2018['FECHAS_ESPECIALES'] = df_gen_2010a2018['FECHAS_ESPECIALES'].fillna('NO ES FECHA ESPECIAL')

In [17]:
df_gen_2010a2018.head()

,FECHA,DEPARTAMENTO_,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE_SITIO,ARMA_EMPLEADA,MOVIL_AGRESOR,...,CANTIDAD,CONFLICTIVIDAD,DELITO,FECHA_FORMATO,AÑO,MES,DIA_NUMERO,SEMANA_NUMERO,DIA_JULIANO,FECHAS_ESPECIALES
0,2010-01-01,ANTIOQUIA,MEDELLÍN (CT),VIERNES,01:00:00,GRANIZAL C-1,RURAL,CASAS DE HABITACION,NO REPORTADO,NO REPORTADO,...,1.0,AMENAZA,NaN,2010-01-01,2010.0,1.0,1.0,53.0,1.0,AÑO NUEVO
1,2010-01-01,ANTIOQUIA,MEDELLÍN (CT),VIERNES,12:00:00,EL RINCON C-16,URBANA,CASAS DE HABITACION,ARMA BLANCA,A PIE,...,1.0,AMENAZA,NaN,2010-01-01,2010.0,1.0,1.0,53.0,1.0,AÑO NUEVO
2,2010-01-01,ARAUCA,TAME,VIERNES,03:30:00,SUCRE,URBANA,VIAS PUBLICAS,NO REPORTADO,CONDUCTOR MOTOCICLETA,...,1.0,AMENAZA,NaN,2010-01-01,2010.0,1.0,1.0,53.0,1.0,AÑO NUEVO
3,2010-01-01,ARAUCA,TAME,VIERNES,06:30:00,CENTRO,URBANA,VIAS PUBLICAS,NO REPORTADO,A PIE,...,1.0,AMENAZA,NaN,2010-01-01,2010.0,1.0,1.0,53.0,1.0,AÑO NUEVO
4,2010-01-01,ATLÁNTICO,BARRANQUILLA (CT),VIERNES,16:30:00,LAS MERCEDES,URBANA,DENTRO DE LA VIVIENDA,NO REPORTADO,A PIE,...,1.0,AMENAZA,NaN,2010-01-01,2010.0,1.0,1.0,53.0,1.0,AÑO NUEVO


In [18]:
df_gen_2010a2018.FECHAS_ESPECIALES.unique()

array(['AÑO NUEVO', 'NO ES FECHA ESPECIAL', 'HALLOWEN', 'DIA VELITAS',
       'NAVIDAD', 'FERIA CALI', 'FIN DE AÑO'], dtype=object)

#### **1.6 AJUSTE DEL FORMATO DE LA COLUMNA ['HORA'] y sus derivadas.** 
> **Nota**
Se derivan las columnas ['HORA_FORMATO_24h'],['HORA_RANGO_24x'].


In [19]:
#Sondeo de datos nulos y NaN 

print('Existe por lo menos algún dato nan: ', df_gen_2010a2018.HORA.isna().any())
print('Existe por lo menos algún dato null: ', df_gen_2010a2018.HORA.isna().any())
print('La cantidad de nan son:', df_gen_2010a2018.HORA.isna().sum())
print('La cantidad de null son:', df_gen_2010a2018.HORA.isnull().sum())

Existe por lo menos algún dato nan:  False
Existe por lo menos algún dato null:  False
La cantidad de nan son: 0
La cantidad de null son: 0


In [20]:
import numpy as np
mask_nan_hora = np.where(df_gen_2010a2018['HORA'].isna())

In [21]:
import datetime
import time

df_gen_2010a2018['HORA_FORMATO_24h']=pd.DatetimeIndex(df_gen_2010a2018['HORA'])
df_gen_2010a2018['HORA_FORMATO_24h'] = [datetime.datetime.time(d) for d in df_gen_2010a2018['HORA_FORMATO_24h']]


In [22]:
#df_gen_2010a2018.info()
from IPython.display import display
display(df_gen_2010a2018.head(2))
display(df_gen_2010a2018.tail(2))

,FECHA,DEPARTAMENTO_,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE_SITIO,ARMA_EMPLEADA,MOVIL_AGRESOR,...,CONFLICTIVIDAD,DELITO,FECHA_FORMATO,AÑO,MES,DIA_NUMERO,SEMANA_NUMERO,DIA_JULIANO,FECHAS_ESPECIALES,HORA_FORMATO_24h
0,2010-01-01,ANTIOQUIA,MEDELLÍN (CT),VIERNES,01:00:00,GRANIZAL C-1,RURAL,CASAS DE HABITACION,NO REPORTADO,NO REPORTADO,...,AMENAZA,NaN,2010-01-01,2010.0,1.0,1.0,53.0,1.0,AÑO NUEVO,01:00:00
1,2010-01-01,ANTIOQUIA,MEDELLÍN (CT),VIERNES,12:00:00,EL RINCON C-16,URBANA,CASAS DE HABITACION,ARMA BLANCA,A PIE,...,AMENAZA,NaN,2010-01-01,2010.0,1.0,1.0,53.0,1.0,AÑO NUEVO,12:00:00


,FECHA,DEPARTAMENTO_,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE_SITIO,ARMA_EMPLEADA,MOVIL_AGRESOR,...,CONFLICTIVIDAD,DELITO,FECHA_FORMATO,AÑO,MES,DIA_NUMERO,SEMANA_NUMERO,DIA_JULIANO,FECHAS_ESPECIALES,HORA_FORMATO_24h
3052532,02/28/2019 12:00:00 AM,VALLE,YUMBO,JUEVES,12/31/1899 02:53:00 PM,PUERTO ISAACS E25,URBANA,VIAS PUBLICAS,NO REPORTADO,A PIE,...,VIOLENCIA INTRAFAMILIAR,NaN,2019-02-28,2019.0,2.0,28.0,9.0,59.0,NO ES FECHA ESPECIAL,14:53:00
3052533,02/28/2019 12:00:00 AM,VICHADA,PUERTO CARREÑO (CT),JUEVES,12/31/1899 01:30:00 PM,CALARCA,URBANA,VIAS PUBLICAS,NO REPORTADO,A PIE,...,VIOLENCIA INTRAFAMILIAR,NaN,2019-02-28,2019.0,2.0,28.0,9.0,59.0,NO ES FECHA ESPECIAL,13:30:00


In [23]:
####################
def hora_24x_(hora):
    
    L=24

    franja_hora_24x = [(x+1) for x in range(int(L))]
    list_hora_inicio = [(60*x) for x in range(int(L))]
    list_hora_final = [(60*x+59) for x in range(int(L))]
    iteracion=0
    
    hora_enminutos = hora.hour*60 + hora.minute
    
    for franja in franja_hora_24x:
        hora_inicio = list_hora_inicio[iteracion]
        hora_final = list_hora_final[iteracion]
        if hora_inicio <= hora_enminutos and hora_enminutos <= hora_final:
            return (franja_hora_24x[iteracion])
        iteracion +=1 
    iteracion=0
####################

In [24]:
df_gen_2010a2018['HORA_24x'] = df_gen_2010a2018.HORA_FORMATO_24h.apply(hora_24x_)

print(sorted(set(df_gen_2010a2018.HORA_24x.unique())))

from IPython.display import display
display(df_gen_2010a2018.tail(2))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


,FECHA,DEPARTAMENTO_,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE_SITIO,ARMA_EMPLEADA,MOVIL_AGRESOR,...,DELITO,FECHA_FORMATO,AÑO,MES,DIA_NUMERO,SEMANA_NUMERO,DIA_JULIANO,FECHAS_ESPECIALES,HORA_FORMATO_24h,HORA_24x
3052532,02/28/2019 12:00:00 AM,VALLE,YUMBO,JUEVES,12/31/1899 02:53:00 PM,PUERTO ISAACS E25,URBANA,VIAS PUBLICAS,NO REPORTADO,A PIE,...,NaN,2019-02-28,2019.0,2.0,28.0,9.0,59.0,NO ES FECHA ESPECIAL,14:53:00,15
3052533,02/28/2019 12:00:00 AM,VICHADA,PUERTO CARREÑO (CT),JUEVES,12/31/1899 01:30:00 PM,CALARCA,URBANA,VIAS PUBLICAS,NO REPORTADO,A PIE,...,NaN,2019-02-28,2019.0,2.0,28.0,9.0,59.0,NO ES FECHA ESPECIAL,13:30:00,14


In [25]:
#df_gen_2010a2018.hora_24x.value_counts().plot(kind='bar', title ="Distribución hora 24x", figsize=(15, 10), legend=True, fontsize=12)

In [26]:
df_gen_2010a2018[['HORA','HORA_FORMATO_24h','HORA_24x']].sample(n=10)

,HORA,HORA_FORMATO_24h,HORA_24x
1000182,11:00:00,11:00:00,12
1062620,05:20:00,05:20:00,6
1135020,15:45:00,15:45:00,16
2534626,13:30:00,13:30:00,14
1165965,20:30:00,20:30:00,21
2392172,16:00:00,16:00:00,17
539938,16:15:00,16:15:00,17
985687,05:00:00,05:00:00,6
1074762,14:40:00,14:40:00,15
2876482,20:30:00,20:30:00,21


In [27]:
df_gen_2010a2018[['HORA','HORA_FORMATO_24h','HORA_24x']].dtypes

HORA                object
HORA_FORMATO_24h    object
HORA_24x             int64
dtype: object

In [28]:
print('numero de casos nan en campo hora: ',df_gen_2010a2018.HORA.isna().sum())

numero de casos nan en campo hora:  0


In [29]:
#Forma excelentepara cambiar valores según otro columna
#df.loc[(pd.isnull(df.C), 'C'] = df.A * df.B
#df_gen_2010a2018['hora_formato_24h'].ix[mask_nan_hora] = 'NO REPORTADO'
#df_gen_2010a2018.loc[pd.isna(df_gen_2010a2018.hora), ['hora_formato_24h','hora_24x']] = np.nan
df_gen_2010a2018.loc[pd.isna(df_gen_2010a2018.HORA), ['HORA_FORMATO_24h','HORA_24x']] = 'SIN DATO'

print('casos NaN en campo hora: ', pd.isna(df_gen_2010a2018.HORA).sum())
print('casos NaN en campo hora_formato_24h: ', pd.isna(df_gen_2010a2018.HORA_FORMATO_24h).sum())
print('casos NaN en campo hora_24x', pd.isna(df_gen_2010a2018.HORA_24x).sum())

casos NaN en campo hora:  0
casos NaN en campo hora_formato_24h:  0
casos NaN en campo hora_24x 0


In [30]:
df_gen_2010a2018[['HORA_FORMATO_24h','HORA_24x']].tail(5)

,HORA_FORMATO_24h,HORA_24x
3052529,00:20:00,1
3052530,07:15:00,8
3052531,19:00:00,20
3052532,14:53:00,15
3052533,13:30:00,14


In [31]:
#print('', df_gen_2010a2018.hora.value_counts())
#print('', df_gen_2010a2018.hora_formato_24h.value_counts())
print('', df_gen_2010a2018.HORA_24x.value_counts())

 1     412252
11    175559
20    170822
21    166733
12    154219
19    142486
16    138476
17    136929
18    134017
10    133544
22    132338
9     131905
13    130056
15    125929
14    112274
23    101897
8      93307
24     82402
7      72519
2      71051
3      67862
4      61996
6      53819
5      50142
Name: HORA_24x, dtype: int64


In [32]:
df_gen_2010a2018.sample(10)

,FECHA,DEPARTAMENTO_,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE_SITIO,ARMA_EMPLEADA,MOVIL_AGRESOR,...,DELITO,FECHA_FORMATO,AÑO,MES,DIA_NUMERO,SEMANA_NUMERO,DIA_JULIANO,FECHAS_ESPECIALES,HORA_FORMATO_24h,HORA_24x
2488750,2018-11-11 00:00:00,ANTIOQUIA,MEDELLÍN (CT),DOMINGO,10:30:00,VDA. LA PALMA S.C.,RURAL,VIAS PUBLICAS,ARMA BLANCA / CORTOPUNZANTE,A PIE,...,NaN,2018-11-11,2018.0,11.0,11.0,45.0,315.0,NO ES FECHA ESPECIAL,10:30:00,11
2524371,2010-02-06 00:00:00,ANTIOQUIA,MEDELLÍN (CT),SÁBADO,10:55:00,CASTILLA C-5,URBANA,VIAS PUBLICAS,CONTUNDENTES,A PIE,...,NaN,2010-02-06,2010.0,2.0,6.0,5.0,37.0,NO ES FECHA ESPECIAL,10:55:00,11
2497538,2018-12-09 00:00:00,ANTIOQUIA,MEDELLÍN (CT),DOMINGO,18:30:00,LA LIBERTAD C-8,URBANA,VIAS PUBLICAS,CONTUNDENTES,A PIE,...,NaN,2018-12-09,2018.0,12.0,9.0,49.0,343.0,NO ES FECHA ESPECIAL,18:30:00,19
835240,2014-07-11 00:00:00,ANTIOQUIA,MEDELLÍN (CT),VIERNES,17:20:00,LA CANDELARIA C-10,URBANA,VIAS PUBLICAS,ARMA BLANCA / CORTOPUNZANTE,A PIE,...,NaN,2014-07-11,2014.0,7.0,11.0,28.0,192.0,NO ES FECHA ESPECIAL,17:20:00,18
1487133,2018-03-06 00:00:00,META,VILLAVICENCIO (CT),DOMINGO,12/31/1899 11:00:00 AM,MESETAS,URBANA,VIAS PUBLICAS,CONTUNDENTES,PASAJERO MOTOCICLETA,...,NaN,2018-03-06,2018.0,3.0,6.0,10.0,65.0,NO ES FECHA ESPECIAL,11:00:00,12
1151917,2016-03-13 00:00:00,SUCRE,SAN LUIS DE SINCÉ,DOMINGO,13:00:00,VEREDA EL NARANJO,RURAL,FINCAS Y SIMILARES,SIN EMPLEO DE ARMAS,A PIE,...,NaN,2016-03-13,2016.0,3.0,13.0,10.0,73.0,NO ES FECHA ESPECIAL,13:00:00,14
2089595,2015-06-16 00:00:00,NORTE DE SANTANDER,VILLA DEL ROSARIO,MARTES,20:15:00,URBANIZACION CAMPO VERDE,URBANA,VIAS PUBLICAS,CONTUNDENTES,A PIE,...,NaN,2015-06-16,2015.0,6.0,16.0,25.0,167.0,NO ES FECHA ESPECIAL,20:15:00,21
1032633,2016-05-20 00:00:00,ANTIOQUIA,MEDELLÍN (CT),VIERNES,22:00:00,PATIO BONITO C-14,URBANA,VIAS PUBLICAS,CONTUNDENTES,A PIE,...,NaN,2016-05-20,2016.0,5.0,20.0,20.0,141.0,NO ES FECHA ESPECIAL,22:00:00,23
720472,2012-01-27 00:00:00,SANTANDER,PUERTO WILCHES,VIERNES,00:00:00,EL CENTRO,URBANA,VIAS PUBLICAS,SIN EMPLEO DE ARMAS,-,...,NaN,2012-01-27,2012.0,1.0,27.0,4.0,27.0,NO ES FECHA ESPECIAL,00:00:00,1
1255405,2017-09-06 00:00:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),MIÉRCOLES,12:00:00,CENTRO INTERNACIONAL E-3,URBANA,VIAS PUBLICAS,SIN EMPLEO DE ARMAS,A PIE,...,NaN,2017-09-06,2017.0,9.0,6.0,36.0,249.0,NO ES FECHA ESPECIAL,12:00:00,13


#### **1.7 AJUSTE DE LA COLUMNA ['SEXO'] y sus derivadas.** 
> **Nota**
Se mapea con el diccionario 
{'MASCULINO':'HOMBRE', 'FEMENINO':'MUJER', '-':'SIN DATO', 'NO REPORTADO':'SIN DATO', 'NO REPORTA':'SIN DATO'}.
En sexo pueden existir tres variables: HOMBRE, MUJER, SIN DATO

In [33]:
print(df_gen_2010a2018.SEXO.unique())
print(df_gen_2010a2018.SEXO.value_counts())

['MASCULINO' 'FEMENINO' 'NO REPORTA' 'NO REPORTADO' '-']
MASCULINO       1539687
FEMENINO        1509662
NO REPORTA         2301
-                   774
NO REPORTADO        110
Name: SEXO, dtype: int64


In [34]:
sexo_diccionario = {'MASCULINO':'HOMBRE', 'FEMENINO':'MUJER', '-':'SIN DATO', 'NO REPORTADO':'SIN DATO', 'NO REPORTA':'SIN DATO'}
df_gen_2010a2018.SEXO = df_gen_2010a2018.SEXO.replace(sexo_diccionario, regex=True)
print(df_gen_2010a2018.SEXO.unique())
print(df_gen_2010a2018.SEXO.value_counts())

['HOMBRE' 'MUJER' 'SIN DATO']
HOMBRE      1539687
MUJER       1509662
SIN DATO       3185
Name: SEXO, dtype: int64


#### **1.8 AJUSTE DE LA COLUMNA ['EDAD'] a formatoi númerico y sus derivadas.** 
> **Nota**
Columnas a derivar ['EDAD_FORMATO'],['EDAD_5Q_80mas'],['EDAD_NNAJ']. Se usan los siguientes criterior para depurar la columna ['EDAD_FORMATO'] 
- ['EDAD_FORMATO'] < 0 = np.nan 
- ['EDAD_FORMATO'] > 115 = np.nan

In [35]:
print('Cantidad de Valores Unicos',len(df_gen_2010a2018.EDAD.unique()))
#print(sorted(set(df_gen_2010a2018.EDAD.unique())))
print(df_gen_2010a2018.EDAD.unique())

Cantidad de Valores Unicos 325
['48' '27' '25' '37' '31' '33' '36' '35' '26' '19' '50' '43' '23' '62'
 'NO REPORTADO' '40' '28' '30' '69' '22' '53' '20' '54' '58' '49' '38'
 '59' '41' '24' '21' '47' '18' '64' '45' '44' '34' '32' '42' '46' '29'
 '66' '55' '56' '57' '39' '61' '17' '15' '16' '52' '60' '51' '67' '77'
 '68' '12' '14' '74' '72' '13' '65' '63' '11' '75' '73' '10' '81' '70'
 '80' '71' '7' '76' '82' '6' '9' '78' '5' '2' '88' '84' '0' '8' '89' '1'
 '4' '83' '79' '86' '3' '92' '90' '-' '85' '91' '87' '-1' '94' '96' '95'
 '18.0' '51.0' '58.0' '36.0' '15.0' '57.0' '28.0' '34.0' '21.0' '22.0'
 '50.0' '53.0' '17.0' '25.0' nan '29.0' '38.0' '26.0' '52.0' '45.0' '27.0'
 '39.0' '42.0' '54.0' '43.0' '23.0' '40.0' '35.0' '62.0' '37.0' '60.0'
 '41.0' '31.0' '19.0' '66.0' '33.0' '63.0' '48.0' '30.0' '20.0' '67.0'
 '32.0' '47.0' '46.0' '2.0' '8.0' '11.0' '59.0' '13.0' '24.0' '56.0'
 '117.0' '44.0' '14.0' '77.0' '55.0' '16.0' '64.0' '49.0' '70.0' '75.0'
 '80.0' '10.0' '61.0' '69.0' '9.0' '1.0

In [36]:
import numpy as np

print('numero de casos con NO', df_gen_2010a2018.EDAD.str.contains('NO').sum())
print('numero de casos con ´-´', df_gen_2010a2018.EDAD.str.contains('-').sum())
print('suma de ambos casos:', df_gen_2010a2018.EDAD.str.contains('NO').sum() + df_gen_2010a2018.EDAD.str.contains('-').sum())

df_gen_2010a2018['EDAD_FORMATO'] = pd.to_numeric(df_gen_2010a2018.EDAD, errors='coerce' )

print(sorted(set(df_gen_2010a2018.EDAD_FORMATO.unique())))
print('items',len(df_gen_2010a2018.EDAD_FORMATO.unique()))

numero de casos con NO 5869
numero de casos con ´-´ 12283
suma de ambos casos: 18152
[nan, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 106.0, 113.0, 117.0, 118.0, 123.0, 226.0, 287.0, 311.0, 316.0, 322.0, 351.0, 378.0, 432.0, 501.0, 701.0, 820.0]
items 121


In [37]:
print(df_gen_2010a2018.EDAD_FORMATO.value_counts())

 18.0     183191
 35.0     145635
 25.0     100125
 26.0      93516
 30.0      92784
 27.0      92431
 28.0      92154
 24.0      91256
 23.0      89694
 22.0      87131
 29.0      87098
 32.0      82345
 21.0      80656
 31.0      78463
 20.0      78200
 33.0      75576
 34.0      74000
 36.0      69887
 19.0      69849
 37.0      65261
 38.0      63017
 40.0      57891
 39.0      56992
 42.0      49375
 41.0      49132
 17.0      46930
 45.0      45403
 43.0      44473
 44.0      42412
 16.0      39628
           ...  
 90.0        231
 91.0        155
 92.0        124
 93.0         69
 94.0         61
-1.0          39
 96.0         36
 95.0         29
 98.0         17
 97.0         16
 100.0         7
 99.0          4
 117.0         4
 118.0         3
 351.0         3
 102.0         2
 701.0         1
 378.0         1
 820.0         1
 316.0         1
 322.0         1
 432.0         1
 311.0         1
 287.0         1
 226.0         1
 123.0         1
 113.0         1
 106.0        

In [38]:
print('Casos nan despues de pasar a numeric {} de {}'.format(df_gen_2010a2018.EDAD_FORMATO.isnull().sum(), len(df_gen_2010a2018.index)))
print('Casos nan despues de pasar a numeric {:.2f} porciento'.format(df_gen_2010a2018.EDAD_FORMATO.isnull().sum()*100/len(df_gen_2010a2018.index)))

Casos nan despues de pasar a numeric 22027 de 3052534
Casos nan despues de pasar a numeric 0.72 porciento


In [39]:
mask = (df_gen_2010a2018['EDAD_FORMATO']<0)
column_name = 'EDAD_FORMATO'
df_gen_2010a2018.loc[mask, column_name] = np.nan

mask = (df_gen_2010a2018['EDAD_FORMATO'] > 115)
column_name = 'EDAD_FORMATO'
df_gen_2010a2018.loc[mask, column_name] = np.nan


In [40]:
print('Casos nan despues de pasar a numeric {} de {}'.format(df_gen_2010a2018.EDAD_FORMATO.isnull().sum(), len(df_gen_2010a2018.index)))
print('Casos nan despues de pasar a numeric {:.2f} porciento'.format(df_gen_2010a2018.EDAD_FORMATO.isnull().sum()*100/len(df_gen_2010a2018.index)))

Casos nan despues de pasar a numeric 22087 de 3052534
Casos nan despues de pasar a numeric 0.72 porciento


In [41]:
print(df_gen_2010a2018.EDAD_FORMATO.value_counts())

18.0     183191
35.0     145635
25.0     100125
26.0      93516
30.0      92784
27.0      92431
28.0      92154
24.0      91256
23.0      89694
22.0      87131
29.0      87098
32.0      82345
21.0      80656
31.0      78463
20.0      78200
33.0      75576
34.0      74000
36.0      69887
19.0      69849
37.0      65261
38.0      63017
40.0      57891
39.0      56992
42.0      49375
41.0      49132
17.0      46930
45.0      45403
43.0      44473
44.0      42412
16.0      39628
          ...  
77.0       2304
1.0        2115
78.0       2047
79.0       1756
80.0       1650
81.0       1240
82.0       1122
0.0        1090
83.0        968
84.0        841
85.0        688
86.0        539
87.0        480
88.0        356
89.0        297
90.0        231
91.0        155
92.0        124
93.0         69
94.0         61
96.0         36
95.0         29
98.0         17
97.0         16
100.0         7
99.0          4
102.0         2
101.0         1
106.0         1
113.0         1
Name: EDAD_FORMATO, Leng

In [42]:
import numpy as np

def edad_quintenal_70mas(edad):
    rangos = 14 
    
    rangos_edad = [(x+1) for x in range(int(rangos))]
    lista_edad_minima = [(5*x) for x in range(int(rangos))]
    lista_edad_maxima = [(5*x + 4) for x in range(int(rangos))]
    
    rangos_edad.extend([15])
    lista_edad_minima.extend([5*14])
    lista_edad_maxima.extend([200])
    iteracion =0
    
    for rango in rangos_edad:
        edad_inicio = lista_edad_minima[iteracion]
        edad_final = lista_edad_maxima[iteracion]
        if edad_inicio <= edad and edad <= edad_final:
            return (str(edad_inicio).zfill(2)+'-'+str(edad_final).zfill(2))
        iteracion +=1 
    iteracion=0
        

def edad_quintenal_80mas(edad):
    rangos = 16 
    
    rangos_edad = [(x+1) for x in range(int(rangos))]
    list_edad_minima = [(5*x) for x in range(int(rangos))]
    list_edad_maxima = [(5*x + 4) for x in range(int(rangos))]
    
    rangos_edad.extend([16])
    list_edad_minima.extend([5*16])
    list_edad_maxima.extend([200])
    
    iteracion =0
   
    for rango in rangos_edad:
        edad_inicio = list_edad_minima[iteracion]
        edad_final = list_edad_maxima[iteracion]
        if edad_inicio <= edad and edad <= edad_final:
            return (str(edad_inicio).zfill(2)+'-'+str(edad_final).zfill(2))
        iteracion +=1 
    iteracion=0

    
def edad_rango9x(edad):
    rangos_edad = [1, 2, 3, 4, 5, 6 ,7, 8, 9]
    list_edad_minima = [0, 5, 10, 15, 18, 25, 30, 45, 60]
    list_edad_maxima  = [4, 9, 14, 17, 24, 29, 44, 59, 200]
    
    iteracion =0
   
    for rango in rangos_edad:
        edad_inicio = list_edad_minima[iteracion]
        edad_final = list_edad_maxima[iteracion]
        if edad_inicio <= edad and edad <= edad_final:
            return (str(edad_inicio).zfill(2)+'-'+str(edad_final).zfill(2))
        iteracion +=1

In [43]:
#df_gen_2010a2018['edad_5Q_70mas'] = df_gen_2010a2018.edad_formato_numero.apply(edad_quintenal_70mas)
df_gen_2010a2018['EDAD_5Q_80mas'] = df_gen_2010a2018.EDAD_FORMATO.apply(edad_quintenal_80mas)
df_gen_2010a2018['EDAD_NNAJ'] = df_gen_2010a2018.EDAD_FORMATO.apply(edad_rango9x)

#df_gen_2010a2018['edad_5Q_70mas'] = df_gen_2010a2018['edad_5Q_70mas'].replace('70-200', '70+')
df_gen_2010a2018['EDAD_5Q_80mas'] = df_gen_2010a2018['EDAD_5Q_80mas'].replace('80-200', '80+')
df_gen_2010a2018['EDAD_NNAJ'] = df_gen_2010a2018['EDAD_NNAJ'].replace('60-200', '60+')

#df_gen_2010a2018['edad_5Q_70mas'] = df_gen_2010a2018['edad_5Q_70mas'].fillna('NO REPORTA')
df_gen_2010a2018['EDAD_5Q_80mas'] = df_gen_2010a2018['EDAD_5Q_80mas'].fillna('SIN DATO')
df_gen_2010a2018['EDAD_NNAJ'] = df_gen_2010a2018['EDAD_NNAJ'].fillna('SIN DATO')


In [44]:
mask = df_gen_2010a2018['EDAD_FORMATO'].isnull()

In [45]:
from IPython.display import display
display(df_gen_2010a2018[mask].sample(10))

,FECHA,DEPARTAMENTO_,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE_SITIO,ARMA_EMPLEADA,MOVIL_AGRESOR,...,MES,DIA_NUMERO,SEMANA_NUMERO,DIA_JULIANO,FECHAS_ESPECIALES,HORA_FORMATO_24h,HORA_24x,EDAD_FORMATO,EDAD_5Q_80mas,EDAD_NNAJ
12695,2011-02-17,NARIÑO,PASTO (CT),JUEVES,11:00:00,CENTRO,URBANA,CASAS DE HABITACION,NO REPORTADO,A PIE,...,2.0,17.0,7.0,48.0,NO ES FECHA ESPECIAL,11:00:00,12,NaN,SIN DATO,SIN DATO
267530,2012-04-03 00:00:00,NARIÑO,PASTO (CT),MARTES,00:00:00,-,RURAL,NO DEFINIDO,NO REPORTADO,A PIE,...,4.0,3.0,14.0,94.0,NO ES FECHA ESPECIAL,00:00:00,1,NaN,SIN DATO,SIN DATO
1802887,2012-03-04 00:00:00,NORTE DE SANTANDER,SILOS,DOMINGO,00:00:00,VEREDA RANCHADERA,RURAL,FINCAS Y SIMILARES,CONTUNDENTES,A PIE,...,3.0,4.0,9.0,64.0,NO ES FECHA ESPECIAL,00:00:00,1,NaN,SIN DATO,SIN DATO
399116,2019-09-01 00:00:00,ANTIOQUIA,YONDÓ,MIÉRCOLES,12/31/1899 08:30:00 PM,VDA. PUERTO TOMAS,URBANA,VIAS PUBLICAS,NO REPORTADO,A PIE,...,9.0,1.0,35.0,244.0,NO ES FECHA ESPECIAL,20:30:00,21,NaN,SIN DATO,SIN DATO
1217104,2017-02-12 00:00:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),DOMINGO,00:00:00,BELALCAZAR E-13,URBANA,VIAS PUBLICAS,SIN EMPLEO DE ARMAS,A PIE,...,2.0,12.0,6.0,43.0,NO ES FECHA ESPECIAL,00:00:00,1,NaN,SIN DATO,SIN DATO
252259,2010-05-22 00:00:00,ANTIOQUIA,BELLO,SÁBADO,16:00:00,NIQUIA CAMACOL,URBANA,CASAS DE HABITACION,NO REPORTADO,A PIE,...,5.0,22.0,20.0,142.0,NO ES FECHA ESPECIAL,16:00:00,17,NaN,SIN DATO,SIN DATO
1686506,2010-03-26 00:00:00,META,VILLAVICENCIO (CT),VIERNES,12:30:00,CALAMAR,URBANA,VIAS PUBLICAS,CONTUNDENTES,A PIE,...,3.0,26.0,12.0,85.0,NO ES FECHA ESPECIAL,12:30:00,13,NaN,SIN DATO,SIN DATO
1738770,2011-03-16 00:00:00,META,VILLAVICENCIO (CT),MIÉRCOLES,21:30:00,ACACIAS,URBANA,VIAS PUBLICAS,CONTUNDENTES,A PIE,...,3.0,16.0,11.0,75.0,NO ES FECHA ESPECIAL,21:30:00,22,NaN,SIN DATO,SIN DATO
270626,2012-07-07 00:00:00,ATLÁNTICO,BARRANQUILLA (CT),SÁBADO,22:30:00,ABAJO,URBANA,VIAS PUBLICAS,NO REPORTADO,NO REPORTADO,...,7.0,7.0,27.0,189.0,NO ES FECHA ESPECIAL,22:30:00,23,NaN,SIN DATO,SIN DATO
2508567,01/15/2019 12:00:00 AM,CUNDINAMARCA,FUNZA,MARTES,12/31/1899 12:00:00 AM,VER. EL CERRITO,URBANA,VIAS PUBLICAS,NO REPORTADO,A PIE,...,1.0,15.0,3.0,15.0,NO ES FECHA ESPECIAL,00:00:00,1,NaN,SIN DATO,SIN DATO


In [46]:
#print(df_gen_2010a2018.edad_5Q_70mas.unique())
print(df_gen_2010a2018.EDAD_5Q_80mas.unique())
print(df_gen_2010a2018.EDAD_NNAJ.unique())

['45-49' '25-29' '35-39' '30-34' '15-19' '50-54' '40-44' '20-24' '60-64'
 'SIN DATO' '65-69' '55-59' '75-79' '10-14' '70-74' '80+' '05-09' '00-04']
['45-59' '25-29' '30-44' '18-24' '60+' 'SIN DATO' '15-17' '10-14' '05-09'
 '00-04']


In [47]:
#df_gen_2010a2018.edad_5q_70mas.hist()
#df_gen_2010a2018.edad_5Q_70mas.value_counts().plot(kind='bar', title ="Distribución edad", figsize=(15, 10), legend=True, fontsize=12)

#### **1.9 AJUSTE DE LA COLUMNA ['BARRIO'] y derivar columnas ['BARRIO_TEMP'], ['ESTACION_COD_DEV'], [COMUNA_DEV].** 
> **Nota**
- ['BARRIO_TEMP'] para esta columna se elimina adiciones como E13, contenidos en (), etc. 
- ['ESTACION_COD_DEV'] se extraer los E13 de cada barrio que lo han excrito
- ['COMUNA DEV'] se deriva de la anterior columna

De un trabajo previo de exploración se han realizado los siguintes hallazgos
- caso 1: -
- caso 2: NO REPORTADO
- caso 3: NO REPORTA
- caso 4: NO REPORTA (A. F. l)
- caso 5: NO REPORTA (S. M. P)
- caso 6: E19NO REPORTADO9 - EXXNO REPORTADOX
- caso 7: Nombre_barrio NO REPORTADO Nombre_barrio

In [48]:
df_gen_2010a2018['BARRIO_TEMP'] = df_gen_2010a2018.BARRIO
print(df_gen_2010a2018['BARRIO_TEMP'].isna().sum())
df_gen_2010a2018.sample(5)

15799


,FECHA,DEPARTAMENTO_,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE_SITIO,ARMA_EMPLEADA,MOVIL_AGRESOR,...,DIA_NUMERO,SEMANA_NUMERO,DIA_JULIANO,FECHAS_ESPECIALES,HORA_FORMATO_24h,HORA_24x,EDAD_FORMATO,EDAD_5Q_80mas,EDAD_NNAJ,BARRIO_TEMP
1294511,2017-06-14 00:00:00,CALDAS,MANIZALES (CT),MIÉRCOLES,16:00:00,ESTAMBUL,URBANA,DENTRO DE LA VIVIENDA,SIN EMPLEO DE ARMAS,A PIE,...,14.0,24.0,165.0,NO ES FECHA ESPECIAL,16:00:00,17,69.0,65-69,60+,ESTAMBUL
2668063,2014-07-27 00:00:00,NORTE DE SANTANDER,CÚCUTA (CT),DOMINGO,22:40:00,BARRIO POLICARPA,URBANA,VIAS PUBLICAS,CONTUNDENTES,A PIE,...,27.0,30.0,208.0,NO ES FECHA ESPECIAL,22:40:00,23,28.0,25-29,25-29,BARRIO POLICARPA
641385,2011-11-22 00:00:00,SUCRE,SINCELEJO (CT),MARTES,10:30:00,PABLO SEXTO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,CONDUCTOR MOTOCICLETA,...,22.0,47.0,326.0,NO ES FECHA ESPECIAL,10:30:00,11,38.0,35-39,30-44,PABLO SEXTO
534588,2010-02-10 00:00:00,CAUCA,PATÍA,MIÉRCOLES,10:00:00,-,URBANA,INTERIOR VEHICULO PARTICULAR,SIN EMPLEO DE ARMAS,A PIE,...,10.0,6.0,41.0,NO ES FECHA ESPECIAL,10:00:00,11,53.0,50-54,45-59,-
135151,2016-05-25,ATLÁNTICO,BARRANQUILLA (CT),MIÉRCOLES,09:10:00,SAN LUIS,URBANA,VIAS PUBLICAS,CONTUNDENTES,A PIE,...,25.0,21.0,146.0,NO ES FECHA ESPECIAL,09:10:00,10,12.0,10-14,10-14,SAN LUIS


In [49]:
## --- Expresiones Regulares ---
#[CE][0-9]{1,2}-[0-9]{1,2}
#[CE][0-9]{1,2}
#[CE]-[0-9]{1,2}
#COMU [0-9]{1,2}
#COMUNA [0-9]{1,2}#
#[0-9]{1,2}-S[0-9]{1,2}-[0-9]{1,2}
#[0-9]{1,2}-S-[0-9]{1,2}-[0-9]{1,2}
#[0-9]{1,2}-S-[0-9]{1,2}
#\([^)]*\)
#[0-9]{1,2}-[0-9]{1,2}

df_gen_2010a2018['BARRIO_TEMP'] = df_gen_2010a2018.BARRIO_TEMP.replace(r'\([^)]*\)|[CE][0-9]{1,2}-[0-9]{1,2}|[CE][0-9]{1,2}|[CE]-[0-9]{1,2}|COMU [0-9]{1,2}','', regex=True)
df_gen_2010a2018['BARRIO_TEMP'] = df_gen_2010a2018.BARRIO_TEMP.replace(r'\([^)]*\)|[CE][0-9]{1,2}-[0-9]{1,2}|[CE][0-9]{1,2}|[CE]-[0-9]{1,2}|COMU [0-9]{1,2}','', regex=True)

lista_a_limpiar = ['NO REPORTADO', 'NO REPORTA', 'SIN ESPECIFICAR']
df_gen_2010a2018['BARRIO_TEMP'] = df_gen_2010a2018.BARRIO_TEMP.replace(lista_a_limpiar,'SIN DATO', regex=True)

df_gen_2010a2018['BARRIO_TEMP'] = df_gen_2010a2018.BARRIO_TEMP.replace(np.nan,'SIN DATO')
df_gen_2010a2018['BARRIO_TEMP'] = df_gen_2010a2018.BARRIO_TEMP.replace('-','SIN DATO')

df_gen_2010a2018['BARRIO_TEMP'] = df_gen_2010a2018.BARRIO_TEMP.str.strip()

In [50]:
print(df_gen_2010a2018['BARRIO_TEMP'].isna().sum())
df_gen_2010a2018[df_gen_2010a2018['BARRIO_TEMP'].str.contains('SIN DATO')==True].sample(5)
#df_gen_2010a2018.BARRIO_TEMP.str.contains('SIN ESPECIFICAR').sum()

0


,FECHA,DEPARTAMENTO_,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE_SITIO,ARMA_EMPLEADA,MOVIL_AGRESOR,...,DIA_NUMERO,SEMANA_NUMERO,DIA_JULIANO,FECHAS_ESPECIALES,HORA_FORMATO_24h,HORA_24x,EDAD_FORMATO,EDAD_5Q_80mas,EDAD_NNAJ,BARRIO_TEMP
25330,2012-02-04,HUILA,NEIVA (CT),SÁBADO,00:00:00,NaN,URBANA,ALMACENES,NO REPORTADO,A PIE,...,4.0,5.0,35.0,NO ES FECHA ESPECIAL,00:00:00,1,47.0,45-49,45-59,SIN DATO
247672,NaN,META,ACACÍAS,MIÉRCOLES,14:18:00,NO REPORTADO,URBANA,RED VIRTUAL O INTERNET,SIN EMPLEO DE ARMAS,A PIE,...,NaN,NaN,NaN,NO ES FECHA ESPECIAL,14:18:00,15,41.0,40-44,30-44,SIN DATO
1738470,2011-03-14 00:00:00,CUNDINAMARCA,GIRARDOT,LUNES,06:30:00,-,URBANA,PERIMETRO URBANO,CONTUNDENTES,NO REPORTADO,...,14.0,11.0,73.0,NO ES FECHA ESPECIAL,06:30:00,7,35.0,35-39,30-44,SIN DATO
1716938,2010-10-24 00:00:00,VALLE,SEVILLA,DOMINGO,11:00:00,-,RURAL,VIAS PUBLICAS,CONTUNDENTES,A PIE,...,24.0,42.0,297.0,NO ES FECHA ESPECIAL,11:00:00,12,60.0,60-64,60+,SIN DATO
789392,2013-06-11 00:00:00,CUNDINAMARCA,VILLETA,MARTES,17:00:00,-,URBANA,VIAS PUBLICAS,CONTUNDENTES,-,...,11.0,24.0,162.0,NO ES FECHA ESPECIAL,17:00:00,18,18.0,15-19,18-24,SIN DATO


In [51]:
df_estacion = pd.DataFrame() 
df_estacion['BARRIO'] = df_gen_2010a2018['BARRIO'].copy()

## --- Expresiones Regulares ---
#[CE][0-9]{1,2}-[0-9]{1,2}
#[CE][0-9]{1,2}
#[CE]-[0-9]{1,2}
#COMU [0-9]{1,2}
#COMUNA [0-9]{1,2}#
#[0-9]{1,2}-S[0-9]{1,2}-[0-9]{1,2}
#[0-9]{1,2}-S-[0-9]{1,2}-[0-9]{1,2}
#[0-9]{1,2}-S-[0-9]{1,2}
#\([^)]*\)
#[0-9]{1,2}-[0-9]{1,2}

#Extraer las REGEX
reg_ex = r'([CE][0-9]{1,2}-[0-9]{1,2}|[CE][0-9]{1,2}|[CE]-[0-9]{1,2}|COMU [0-9]{1,2}|COMUNA [0-9]{1,2}|[0-9]{1,2}-S[0-9]{1,2}-[0-9]{1,2}|[0-9]{1,2}-S-[0-9]{1,2}-[0-9]{1,2}|[0-9]{1,2}-S-[0-9]{1,2}|\([^)]*\)|[0-9]{1,2}-[0-9]{1,2})'
df_estacion['ESTACION'] = df_estacion['BARRIO'].str.extract(reg_ex, expand=True)

print ('Casos cambiados', df_estacion.ESTACION.notna().sum()) 

df_estacion['ESTACION'] = df_estacion['ESTACION'].replace('NO REPORTADO', '-', regex =  True)
df_estacion.ESTACION.unique()


Casos cambiados 1107951


array(['C-1', 'C-16', nan, 'E-3', 'E-7', 'E-8', 'E-18', 'E-2', 'E-4',
       'E14', 'E15', 'E7', 'E18', 'E25', 'E6', 'E-19', 'E-10', 'E-15',
       '(URB. SAN BUENAVENTURA)', 'E5', 'E20', 'C-15', 'E-14', 'E11',
       'E4', 'E19', 'E3', 'E-12', 'E-11', 'E2', 'E1', 'E10', 'C-14',
       'E-9', 'E-5', 'E-13', '(SECTOR II )', 'C-5', 'E-1', 'E12', 'E-16',
       'E9', 'E21', 'E8', 'E13', 'E22', 'E24', 'E-17', 'E16', '1-2',
       '(TEORAMA)', 'E-6', 'C-6', '(EL MERCADO)', '( LA TORRE )', 'E17',
       'C-13', '( VILLA LINA )', '(TREBOL)', '(ROZO)', 'E20-5', '(CAUCA)',
       '(19-4)', 'C-11', 'C-10', '(23-1 23-2)', '(A)', '(25-2)',
       '(LETICIA)', 'C-4', '(COMUNA 2)', '(COMUNA 11)', '(SALADO)',
       '(C-2)', '(BOCHALEMA)', '(24-S-01)', '(23-S-2-1)', '(23-2)',
       '(EL ZULIA)', '(C-5)', 'E19-9', '(COLORADOS)', 'E18-1', 'C-3',
       '(PTO CALDAS)', '(C-1)', '(CONVENCION)', 'E19-1', '(LOS SANTOS)',
       '(MARQUETALIA)', '(24-S-02)', '(LABATECA)', '(SAN CALIXTO)',
       '(SECTOR I

In [52]:
df_estacion.sample(10)

,BARRIO,ESTACION
2754874,SAN LUIS E6,E6
532591,CENTRAL DE ABASTOS,NaN
2187605,CRISTO REY,NaN
2503228,PARQUE PRINCIPAL,NaN
102556,NELSON MANDELA,NaN
1004467,LA SALLE,NaN
2598539,LOS ALMENDROS,NaN
2347421,VIDA EN PRIMAVERA,NaN
1273082,EL SALITRE E-13,E-13
2274126,EL MIRADOR E-10,E-10


In [53]:
df_estacion[df_estacion['BARRIO'].str.contains('\(')==True].sample(10)

,BARRIO,ESTACION
3037239,MANTE CLARO (ROZO),(ROZO)
2822506,EL JARDIN (C-4),(C-4)
2195156,JUAN XXIII (C-2),(C-2)
2116547,SECTOR INDUSTRIAL ACOPI (25-2),(25-2)
830506,CANGREJO (26-2),(26-2)
282616,BARRIO CENTRO (CUCUTILLA),(CUCUTILLA)
2325979,LAS PALMAS (C-3),(C-3)
724387,INTERLAKEN (1),(1)
1170062,MANTE CLARO (ROZO),(ROZO)
56525,URB. POBLADO CAMPESTRE (23-S-6-3),(23-S-6-3)


In [54]:
reg_ex = r'([CE][0-9]{1,2}-[0-9]{1,2}|[CE][0-9]{1,2}|[CE]-[0-9]{1,2}|COMU [0-9]{1,2}|COMUNA [0-9]{1,2}|[0-9]{1,2}-S[0-9]{1,2}-[0-9]{1,2}|[0-9]{1,2}-S-[0-9]{1,2}-[0-9]{1,2}|[0-9]{1,2}-S-[0-9]{1,2}|[0-9]{1,2}-[0-9]{1,2})'
df_estacion['ESTACION_1'] = df_estacion['ESTACION'].str.extract(reg_ex, expand=True)

print ('Casos cambiados', df_estacion.ESTACION_1.notna().sum()) 

df_estacion['ESTACION_1'] = df_estacion['ESTACION_1'].fillna('SIN DATO')


Casos cambiados 1093281


In [55]:
df_estacion[df_estacion['BARRIO'].str.contains('\(')==True].sample(10)
#df_estacion.sample(10)

,BARRIO,ESTACION,ESTACION_1
101320,CENTRO (LETICIA),(LETICIA),SIN DATO
2102690,EL PEDREGAL (URB.),(URB.),SIN DATO
144416,CENTRO (C-1),(C-1),C-1
2501803,MANTE CLARO (ROZO),(ROZO),SIN DATO
2634468,OBRERO (23-2),(23-2),23-2
2739440,CORREG. (PTO CALDAS),(PTO CALDAS),SIN DATO
2738699,VEREDA GALAN (VILLA DEL ROSARIO),(VILLA DEL ROSARIO),SIN DATO
2112998,20 DE JULIO (23-S-2-3),(23-S-2-3),23-S-2-3
2006333,CENTRO (A) PEREIRA,(A),SIN DATO
2003963,VEREDA PALO GORDO (VILLA DEL ROSARIO),(VILLA DEL ROSARIO),SIN DATO


In [56]:
df_estacion.ESTACION_1.unique()

array(['C-1', 'C-16', 'SIN DATO', 'E-3', 'E-7', 'E-8', 'E-18', 'E-2',
       'E-4', 'E14', 'E15', 'E7', 'E18', 'E25', 'E6', 'E-19', 'E-10',
       'E-15', 'E5', 'E20', 'C-15', 'E-14', 'E11', 'E4', 'E19', 'E3',
       'E-12', 'E-11', 'E2', 'E1', 'E10', 'C-14', 'E-9', 'E-5', 'E-13',
       'C-5', 'E-1', 'E12', 'E-16', 'E9', 'E21', 'E8', 'E13', 'E22',
       'E24', 'E-17', 'E16', '1-2', 'E-6', 'C-6', 'E17', 'C-13', 'E20-5',
       '19-4', 'C-11', 'C-10', '23-1', '25-2', 'C-4', 'COMUNA 2',
       'COMUNA 11', 'C-2', '24-S-01', '23-S-2-1', '23-2', 'E19-9',
       'E18-1', 'C-3', 'E19-1', '24-S-02', '25-13', 'C-9', 'COMUNA 15',
       'E27', '23-S-2-3', '23-S-3-1', 'COMUNA 8', 'E18-5', '23-S-6-3',
       'C-7', '25-10', '26-2', '27-1', '26-1', 'COMU 10', '27-3', '11-17',
       '23-4', '23-3', '24-S2-1', '24-S1-1', 'COMU 3', '27-2', '23-S-6-1',
       'E26', '23-S-4-2', 'C-8', 'C-12', '25-1', '23-S-1-1', '1-4',
       '8-10', '25-8', '23-S-2-2', 'COMUNA 10', '23-S-4-1', 'E-20',
       'COMUN

In [57]:
df_estacion['ESTACION_COD_DEV'] = 'E-'+ df_estacion['ESTACION_1'].str.extract('([0-9]{1,2})', expand=True )
df_estacion['ESTACION_COD_DEV'] = df_estacion['ESTACION_COD_DEV'].fillna('SIN DATO')

In [58]:
df_estacion.ESTACION_COD_DEV.unique()

array(['E-1', 'E-16', 'SIN DATO', 'E-3', 'E-7', 'E-8', 'E-18', 'E-2',
       'E-4', 'E-14', 'E-15', 'E-25', 'E-6', 'E-19', 'E-10', 'E-5',
       'E-20', 'E-11', 'E-12', 'E-9', 'E-13', 'E-21', 'E-22', 'E-24',
       'E-17', 'E-23', 'E-27', 'E-26', 'E-67', 'E-32', 'E-66'],
      dtype=object)

In [59]:
df_gen_2010a2018['ESTACION_COD_DEV'] = df_estacion['ESTACION_COD_DEV']

In [60]:
df_gen_2010a2018.ESTACION_COD_DEV.unique()

array(['E-1', 'E-16', 'SIN DATO', 'E-3', 'E-7', 'E-8', 'E-18', 'E-2',
       'E-4', 'E-14', 'E-15', 'E-25', 'E-6', 'E-19', 'E-10', 'E-5',
       'E-20', 'E-11', 'E-12', 'E-9', 'E-13', 'E-21', 'E-22', 'E-24',
       'E-17', 'E-23', 'E-27', 'E-26', 'E-67', 'E-32', 'E-66'],
      dtype=object)

In [61]:
df_gen_2010a2018['COMUNA_DEV'] = df_gen_2010a2018['ESTACION_COD_DEV'].str.extract('([0-9]{1,2})', expand=True )
df_gen_2010a2018['COMUNA_DEV'] = df_gen_2010a2018['COMUNA_DEV'].str.zfill(2)
df_gen_2010a2018['COMUNA_DEV'] = df_gen_2010a2018['COMUNA_DEV'].fillna('SIN DATO')

In [62]:
df_gen_2010a2018.COMUNA_DEV.unique()

array(['01', '16', 'SIN DATO', '03', '07', '08', '18', '02', '04', '14',
       '15', '25', '06', '19', '10', '05', '20', '11', '12', '09', '13',
       '21', '22', '24', '17', '23', '27', '26', '67', '32', '66'],
      dtype=object)

In [63]:
df_gen_2010a2018.sample(10)

,FECHA,DEPARTAMENTO_,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE_SITIO,ARMA_EMPLEADA,MOVIL_AGRESOR,...,DIA_JULIANO,FECHAS_ESPECIALES,HORA_FORMATO_24h,HORA_24x,EDAD_FORMATO,EDAD_5Q_80mas,EDAD_NNAJ,BARRIO_TEMP,ESTACION_COD_DEV,COMUNA_DEV
606785,2011-08-09 00:00:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),MARTES,19:15:00,GERMANIA E-3,URBANA,INTERIOR VEHICULO SERVICIO PUBLICO,ARMA BLANCA / CORTOPUNZANTE,PASAJERO BUS,...,221.0,NO ES FECHA ESPECIAL,19:15:00,20,26.0,25-29,25-29,GERMANIA,E-3,03
304442,2015-03-21 00:00:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),SÁBADO,17:00:00,CIUDAD BERNA E-15,URBANA,VIAS PUBLICAS,ESCOPOLAMINA,A PIE,...,80.0,NO ES FECHA ESPECIAL,17:00:00,18,13.0,10-14,10-14,CIUDAD BERNA,E-15,15
2174323,2016-02-28 00:00:00,CAUCA,TIMBÍO,DOMINGO,01:00:00,BELEN,URBANA,VIAS PUBLICAS,ARMA BLANCA / CORTOPUNZANTE,A PIE,...,59.0,NO ES FECHA ESPECIAL,01:00:00,2,20.0,20-24,18-24,BELEN,SIN DATO,SIN DATO
1094694,2016-04-21 00:00:00,BOLÍVAR,CARTAGENA (CT),JUEVES,16:30:00,MEMBRILLAL,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,PASAJERO MOTOCICLETA,...,112.0,NO ES FECHA ESPECIAL,16:30:00,17,43.0,40-44,30-44,MEMBRILLAL,SIN DATO,SIN DATO
1447975,2018-10-04 00:00:00,META,VILLAVICENCIO (CT),MARTES,12/31/1899 01:30:00 PM,VAINILLA,URBANA,VIAS PUBLICAS,SIN EMPLEO DE ARMAS,A PIE,...,277.0,NO ES FECHA ESPECIAL,13:30:00,14,32.0,30-34,30-44,VAINILLA,SIN DATO,SIN DATO
1482921,05/29/2018 12:00:00 AM,CUNDINAMARCA,BOGOTÁ D.C. (CT),MARTES,12/31/1899 07:30:00 AM,MAZUREN I SECTOR E-11,URBANA,BUS TRANSMILENIO,SIN EMPLEO DE ARMAS,A PIE,...,149.0,NO ES FECHA ESPECIAL,07:30:00,8,29.0,25-29,25-29,MAZUREN I SECTOR,E-11,11
2182105,2016-03-08 00:00:00,CUNDINAMARCA,GACHANCIPÁ,MARTES,15:30:00,VER.SAN MARTIN,RURAL,VIAS PUBLICAS,CONTUNDENTES,A PIE,...,68.0,NO ES FECHA ESPECIAL,15:30:00,16,44.0,40-44,30-44,VER.SAN MARTIN,SIN DATO,SIN DATO
374707,2018-01-31 00:00:00,HUILA,NEIVA (CT),MIÉRCOLES,13:00:00,VILLA MARCELA ETAPA I,URBANA,CASAS DE HABITACION,SIN EMPLEO DE ARMAS,A PIE,...,31.0,NO ES FECHA ESPECIAL,13:00:00,14,6.0,05-09,05-09,VILLA MARCELA ETAPA I,SIN DATO,SIN DATO
1449064,2018-12-04 00:00:00,BOLÍVAR,CARTAGENA (CT),JUEVES,12/31/1899 10:00:00 AM,PIE DE LA POPA,URBANA,ALMACENES,SIN EMPLEO DE ARMAS,A PIE,...,338.0,NO ES FECHA ESPECIAL,10:00:00,11,28.0,25-29,25-29,PIE DE LA POPA,SIN DATO,SIN DATO
1042851,2016-06-26 00:00:00,ANTIOQUIA,BELLO,DOMINGO,02:00:00,LA CABAÑA,URBANA,VIAS PUBLICAS,SIN EMPLEO DE ARMAS,PASAJERO MOTOCICLETA,...,178.0,NO ES FECHA ESPECIAL,02:00:00,3,31.0,30-34,30-44,LA CABAÑA,SIN DATO,SIN DATO


In [64]:
import datetime
now = datetime.datetime.now()
ruta_1 = './df_gen_2010u2019_nal/'
file_name ='df_gen_2010u2019_nacional_'

nombre_archivo = file_name + str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'.csv'

df_gen_2010a2018.reset_index(drop=True)
df_gen_2010a2018.to_csv((ruta_1+nombre_archivo), index = False)

print("El df_gen_2010a2018 nacional se ha guardado con el nombre {}".format(nombre_archivo))
print('Tamaño del dataframe final es: ', df_gen_2010a2018.shape)


El df_gen_2010a2018 nacional se ha guardado con el nombre df_gen_2010u2019_nacional_20190421.csv
Tamaño del dataframe final es:  (3052534, 37)


### **ETAPA 2. TRATAMIENTO PARA EL CASO ['CIUDAD'] CALI(CT)**


#### **2.1 SELECCION CASOS CALI (CT)** 




In [65]:
# Se identificaran los casos asociados a la ciudad de CALI
#Inicialmente buscamos dentro de la columna municipio cuales es contiene la palabra CALI
#Para detectar las formas como se refieren a este municipio

#Hacemos una mascara para filtar los datos con CALI

index_casos_con_cali = df_gen_2010a2018['MUNICIPIO'].str.contains('CALI')
casos_con_cali_counts =df_gen_2010a2018['MUNICIPIO'][index_casos_con_cali].value_counts()
print(casos_con_cali_counts)


CALI (CT)      217449
CALIMA            945
SAN CALIXTO       170
CALIFORNIA        114
Name: MUNICIPIO, dtype: int64


In [66]:
index_casos_cali = df_gen_2010a2018['MUNICIPIO']=='CALI (CT)'

df_gen_2010a2018_cali = pd.DataFrame()
df_gen_2010a2018_cali = df_gen_2010a2018[index_casos_cali].copy()
df_gen_2010a2018_cali = df_gen_2010a2018_cali.reset_index(drop=True)
df_gen_2010a2018_cali.sample(5)

,FECHA,DEPARTAMENTO_,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE_SITIO,ARMA_EMPLEADA,MOVIL_AGRESOR,...,DIA_JULIANO,FECHAS_ESPECIALES,HORA_FORMATO_24h,HORA_24x,EDAD_FORMATO,EDAD_5Q_80mas,EDAD_NNAJ,BARRIO_TEMP,ESTACION_COD_DEV,COMUNA_DEV
195256,2013-10-01 00:00:00,VALLE,CALI (CT),MARTES,17:00:00,EL RETIRO E15,URBANA,DENTRO DE LA VIVIENDA,CONTUNDENTES,A PIE,...,274.0,NO ES FECHA ESPECIAL,17:00:00,18,18.0,15-19,18-24,EL RETIRO,E-15,15
144115,2013-04-19 00:00:00,VALLE,CALI (CT),VIERNES,09:00:00,EL POBLADO II E13,URBANA,VIAS PUBLICAS,MEDICAMENTOS,A PIE,...,109.0,NO ES FECHA ESPECIAL,09:00:00,10,38.0,35-39,30-44,EL POBLADO II,E-13,13
192175,2012-10-31 00:00:00,VALLE,CALI (CT),MIÉRCOLES,08:00:00,EL PONDAJE E13,URBANA,CASAS DE HABITACION,CONTUNDENTES,A PIE,...,305.0,HALLOWEN,08:00:00,9,25.0,25-29,25-29,EL PONDAJE,E-13,13
116409,06/21/2018 12:00:00 AM,VALLE,CALI (CT),JUEVES,12/31/1899 01:00:00 PM,SANTA MONICA BELALCAZAR E9,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,CONDUCTOR MOTOCICLETA,...,172.0,NO ES FECHA ESPECIAL,13:00:00,14,47.0,45-49,45-59,SANTA MONICA BELALCAZAR,E-9,09
3405,2014-02-03,VALLE,CALI (CT),LUNES,00:00:00,CGTO LA LEONERA E1,URBANA,VIAS PUBLICAS,SIN EMPLEO DE ARMAS,A PIE,...,34.0,NO ES FECHA ESPECIAL,00:00:00,1,56.0,55-59,45-59,CGTO LA LEONERA,E-1,01


#### **2.2 LIMPIEZA ['BARRIO_TEMP'] y derivar columnas ** 
> **Nota**
- ['BARRIO_TEMP']
- ['BARRIO_OFICIAL'] . Por medio de un mapeo 
- ['CORREGIMIENTO'] 
- ['VEREDA']

In [67]:
def gerate_df_from_valuecounts(value_counts, list_names_columns):
    values_counts_ = value_counts 
    dict_values_counts_ = value_counts.to_dict()
    
    df= pd.DataFrame(list(dict_values_counts_.items()), columns=list_names_columns)
    df = df.sort_values(list_names_columns[0])
    df = df.reset_index(drop=True)
    print('El dataframe tiene tamaño de: ', df.shape)
    
    return df

In [68]:
print('Valores unicos para nombres de barrios {} \n'.format(len(df_gen_2010a2018_cali.BARRIO_TEMP.unique())))
print(df_gen_2010a2018_cali.BARRIO_TEMP.unique())
print()

Valores unicos para nombres de barrios 526 

['ALFONSO B. ARAGON' 'VALLADO' 'SIETE DE AGOSTO' 'CALDAS'
 'CIUDADELA FLORALIA' 'CHIMINANGOS II' 'BELISARIO CAICEDO'
 'JOSE MARIA CORDOBA' 'POPULAR' 'SAN FERNANDO NUEVO' 'TEQUENDAMA'
 'LA RIVERA' 'SAN PEDRO' 'CIUDAD MODELO' 'JORGE ELIECER GAITAN'
 'EL BOSQUE' 'ALTO AGUACATAL' 'LAS GRANJAS' 'LOS ANDES' 'PORVENIR'
 'NUEVA FLORESTA' 'SANTA BARBARA' 'URB. COLSEGUROS' 'ALTO NAPOLES'
 'EL GUABAL' 'PUERTO MALLARINO' 'EL RETIRO' 'LOS FARALLONES' 'LA CASCADA'
 'SILOE' 'BELLAVISTA' 'ALAMEDA' 'BRISAS DE LOS ALAMOS'
 'URBANIZACION DESEPAZ' 'PRADOS DEL SUR' 'CHAPINERO' 'PUERTAS DEL SOL'
 'SUCRE' 'ALTO MELENDEZ' 'ALFONSO LOPEZ II' 'LOS COMUNEROS II' 'PANCE'
 'VISTAHERMOSA' 'PETECUY II' 'CGTO MONTEBELLO' 'CIUDAD CORDOBA'
 'TORRES DE CONFANDI' 'SAN LUIS' 'GUAYAQUIL' 'LOS LAGOS' 'VILLANUEVA'
 'VIPASA' 'INV. CINTA SARDI' 'LOS PARQUES' 'VERSALLES' 'ALFONSO LOPEZ I'
 'SAN VICENTE' 'PACARA' 'PRIMAVERA' 'UNION DE VIVIENDA POPULAR'
 'SANTA ISABEL' 'REPUBLICA ISRAE

In [69]:
df_barrios_fase1 = gerate_df_from_valuecounts(df_gen_2010a2018_cali.BARRIO.value_counts(), ['barrio', 'conteo'])
df_barrios_fase2 = gerate_df_from_valuecounts(df_gen_2010a2018_cali.BARRIO_TEMP.value_counts(), ['barrio', 'conteo'])


El dataframe tiene tamaño de:  (558, 2)
El dataframe tiene tamaño de:  (526, 2)


In [70]:
df_barrios_fase2

,barrio,conteo
0,12 DE OCTUBRE,372
1,20 DE JULIO,220
2,3 DE JULIO,362
3,ACUEDUCTO SAN ANTONIO,53
4,AGUABLANCA,437
5,AGUACATAL,718
6,ALAMEDA,845
7,ALFEREZ REAL,202
8,ALFONSO 2,3
9,ALFONSO 3,10


#### DERIVAR COLUMNA ['BARRIO_NOMBRE']

In [71]:
df_gen_2010a2018_cali['BARRIO_NOMBRE'] = df_gen_2010a2018_cali['BARRIO_TEMP']

#### DERIVAR COLUMNA CORREGIMIENTO 

In [72]:
df_gen_2010a2018_cali['BARRIO_NOMBRE'][df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains('CGTO')].value_counts()

CGTO LA LEONERA          3062
CGTO MONTEBELLO           842
CGTO LA BUITRERA          436
CGTO. LA SIRENA           338
CGTO HORMIGUERO           305
CGTO GOLONDRINAS          181
CGTO NAVARRO              157
CGTO LA VORAGINE          141
CGTO FELIDIA              131
CGTO LA REFORMA           113
CGTO SALADITO              92
CGTO CAMPO ALEGRE          79
CGTO PICHINDE              78
CGTO LOS ANDES             71
CGTO PANCE                 64
CGTO ELVIRA                61
CGTO LAS PALMAS            48
CGTO CASTILLA              26
CGTO LA FONDA              21
CGTO VILLACARMELO          20
CGTO LA PAZ                15
CGTO PEÑAS BLANCAS          3
CGTO PASO DE LA BOLSA       1
CGTO LA LIBERIA             1
CGTO POTRERITO              1
CGTO JUANCHITO              1
CGTO AMPUDIA                1
CGTO SAN ANTONIO            1
Name: BARRIO_NOMBRE, dtype: int64

In [73]:
print(df_gen_2010a2018_cali.BARRIO_TEMP.str.contains("CGTO").sum())
print(df_gen_2010a2018_cali.BARRIO_NOMBRE.str.contains("CGTO").sum())
df_gen_2010a2018_cali['BARRIO_NOMBRE'][df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains('CGTO')].value_counts()

6290
6290


CGTO LA LEONERA          3062
CGTO MONTEBELLO           842
CGTO LA BUITRERA          436
CGTO. LA SIRENA           338
CGTO HORMIGUERO           305
CGTO GOLONDRINAS          181
CGTO NAVARRO              157
CGTO LA VORAGINE          141
CGTO FELIDIA              131
CGTO LA REFORMA           113
CGTO SALADITO              92
CGTO CAMPO ALEGRE          79
CGTO PICHINDE              78
CGTO LOS ANDES             71
CGTO PANCE                 64
CGTO ELVIRA                61
CGTO LAS PALMAS            48
CGTO CASTILLA              26
CGTO LA FONDA              21
CGTO VILLACARMELO          20
CGTO LA PAZ                15
CGTO PEÑAS BLANCAS          3
CGTO PASO DE LA BOLSA       1
CGTO LA LIBERIA             1
CGTO POTRERITO              1
CGTO JUANCHITO              1
CGTO AMPUDIA                1
CGTO SAN ANTONIO            1
Name: BARRIO_NOMBRE, dtype: int64

In [74]:
#Corregimiento
mask_cgto = (df_gen_2010a2018_cali.BARRIO_NOMBRE.str.contains("CGTO"))

df_gen_2010a2018_cali['CGTO_NOMBRE'] = df_gen_2010a2018_cali.BARRIO_NOMBRE[mask_cgto]
df_gen_2010a2018_cali['CGTO_NOMBRE'] = df_gen_2010a2018_cali.CGTO_NOMBRE.replace({"CGTO. ":"", "CGTO ":""},regex=True)
df_gen_2010a2018_cali['CGTO_NOMBRE'] = df_gen_2010a2018_cali.CGTO_NOMBRE.fillna('SIN DATO')

#Limpiar de barrio_nombre los casos detectados
df_gen_2010a2018_cali['BARRIO_NOMBRE'] = df_gen_2010a2018_cali.BARRIO_NOMBRE.replace(to_replace = df_gen_2010a2018_cali.BARRIO_NOMBRE[mask_cgto], value="CORREGIMIENTO", regex=True)



In [75]:
from IPython.display import display
display(df_gen_2010a2018_cali[['BARRIO_NOMBRE','CGTO_NOMBRE']][mask_cgto].sample(10))

,BARRIO_NOMBRE,CGTO_NOMBRE
217366,CORREGIMIENTO,LA LEONERA
13576,CORREGIMIENTO,LA LEONERA
79562,CORREGIMIENTO,MONTEBELLO
178051,CORREGIMIENTO,LA REFORMA
20415,CORREGIMIENTO,LA LEONERA
16529,CORREGIMIENTO,LA LEONERA
13597,CORREGIMIENTO,LA LEONERA
54407,CORREGIMIENTO,LA VORAGINE
142974,CORREGIMIENTO,LA BUITRERA
172507,CORREGIMIENTO,LA SIRENA


In [76]:
df_barrios_fase3 = gerate_df_from_valuecounts(df_gen_2010a2018_cali.BARRIO_NOMBRE.value_counts(), ['barrio', 'conteo'])

El dataframe tiene tamaño de:  (499, 2)


#### DERIVAR COLUMNA VEREDA 

In [77]:
#VEREDA
mask_vereda = (df_gen_2010a2018_cali.BARRIO_NOMBRE.str.contains('VEREDA') | 
               df_gen_2010a2018_cali.BARRIO_NOMBRE.str.contains('VDA') )

df_gen_2010a2018_cali['VEREDA_NOMBRE'] = df_gen_2010a2018_cali.BARRIO_NOMBRE[mask_vereda]
df_gen_2010a2018_cali['VEREDA_NOMBRE'] = df_gen_2010a2018_cali.VEREDA_NOMBRE.replace({"VEREDA ":""},regex=True)

#Limpiar de barrio_nombre los casos detectados
df_gen_2010a2018_cali['BARRIO_NOMBRE'] = df_gen_2010a2018_cali.BARRIO_NOMBRE.replace(to_replace = df_gen_2010a2018_cali.BARRIO_NOMBRE[mask_vereda], value="VEREDA", regex=True)


In [78]:
from IPython.display import display
display(df_gen_2010a2018_cali[['ZONA','BARRIO_NOMBRE','CGTO_NOMBRE', 'VEREDA_NOMBRE']][mask_vereda].sample(10))

,ZONA,BARRIO_NOMBRE,CGTO_NOMBRE,VEREDA_NOMBRE
169915,URBANA,VEREDA,SIN DATO,CASCAJAL
200471,RURAL,VEREDA,SIN DATO,SAN FRANCISCO
164503,URBANA,VEREDA,SIN DATO,CASCAJAL
33368,RURAL,VEREDA,SIN DATO,MONTAÑITAS
105586,RURAL,VEREDA,SIN DATO,MONTAÑITAS
14684,URBANA,VEREDA,SIN DATO,CASCAJAL
162322,RURAL,VEREDA,SIN DATO,LA CASTELLANA
149020,RURAL,VEREDA,SIN DATO,MONTAÑITAS
10952,RURAL,VEREDA,SIN DATO,EL PEON
71727,RURAL,VEREDA,SIN DATO,CASCAJAL


In [79]:
from IPython.display import display
mask_prueba = (df_gen_2010a2018_cali.BARRIO_NOMBRE == 'LOS ANDES') & (df_gen_2010a2018_cali.ZONA == 'RURAL')
print(len(mask_prueba))
display(df_gen_2010a2018_cali[['ZONA','BARRIO_NOMBRE','CGTO_NOMBRE', 'VEREDA_NOMBRE']][mask_prueba].sample(1))

217449


,ZONA,BARRIO_NOMBRE,CGTO_NOMBRE,VEREDA_NOMBRE
99203,RURAL,LOS ANDES,SIN DATO,NaN


In [80]:
sorted(set(df_gen_2010a2018_cali['BARRIO_NOMBRE'].unique()))

['12 DE OCTUBRE',
 '20 DE JULIO',
 '3 DE JULIO',
 'ACUEDUCTO SAN ANTONIO',
 'AGUABLANCA',
 'AGUACATAL',
 'ALAMEDA',
 'ALFEREZ REAL',
 'ALFONSO 2',
 'ALFONSO 3',
 'ALFONSO B. ARAGON',
 'ALFONSO BARBERENA',
 'ALFONSO LOPEZ I',
 'ALFONSO LOPEZ II',
 'ALFONSO LOPEZ III',
 'ALIRIO MORA BELTRAN',
 'ALTO AGUACATAL',
 'ALTO JORDAN',
 'ALTO MELENDEZ',
 'ALTO NAPOLES',
 'ALTO SANTA ISABEL',
 'ALTOS DE MENGA',
 'ALTOS NORMANDIA',
 'ANDRES SANIN',
 'ANGEL DEL HOGAR',
 'ANTONIO NARIÑO',
 'ARANJUEZ',
 'ARBOLEDAS',
 'ASENTAMIENTO LA PLAYITA',
 'ASPROSOCIAL - DIAMANTE',
 'ASTURIAS',
 'ATANASIO GIRARDOT',
 'BAJO AGUACATAL',
 'BAJOS CIUDAD CORDOBA',
 'BARRANQUILLA',
 'BASE AEREA',
 'BATALLON PICHINCHA',
 'BELALCAZAR',
 'BELEN',
 'BELISARIO CAICEDO',
 'BELLA  SUIZA',
 'BELLA SUIZA',
 'BELLA VISTA',
 'BELLAVISTA',
 'BELLO HORIZONTE',
 'BENJAMIN HERRERA',
 'BOLIVARIANO',
 'BOSQUE MUNICIPAL',
 'BOSQUES DEL LIMONAR',
 'BRETAÑA',
 'BRISAS DE LOS ALAMOS',
 'BRISAS DE MAYO',
 'BRISAS DEL GUAVITO',
 'BRISAS DEL 

#### DERIVAR COLUMNA INVASION

In [81]:
#INVASION
mask_invasion = (df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("INV. ")) | (df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("INV ")) | (df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("INVASION "))
print(mask_invasion.sum())

df_gen_2010a2018_cali['INVASION_NOMBRE'] = df_gen_2010a2018_cali.BARRIO_NOMBRE[mask_invasion]
df_gen_2010a2018_cali['INVASION_NOMBRE'] = df_gen_2010a2018_cali.INVASION_NOMBRE.replace({"INV ":"", 'INV. ':'','INVASION':''},regex=True)

#Limpiar de barrio_nombre los casos detectados
df_gen_2010a2018_cali['BARRIO_NOMBRE'] = df_gen_2010a2018_cali.BARRIO_NOMBRE.replace(to_replace = df_gen_2010a2018_cali.BARRIO_NOMBRE[mask_invasion], value="INVASION", regex=True)


993


In [82]:
from IPython.display import display
display(df_gen_2010a2018_cali[['ZONA','BARRIO_NOMBRE','CGTO_NOMBRE', 'VEREDA_NOMBRE', 'INVASION_NOMBRE']][mask_invasion].sample(10))

,ZONA,BARRIO_NOMBRE,CGTO_NOMBRE,VEREDA_NOMBRE,INVASION_NOMBRE
196256,RURAL,INVASION,SIN DATO,NaN,CINTA SARDI
33230,URBANA,INVASION,SIN DATO,NaN,CINTA SARDI
27958,URBANA,INVASION,SIN DATO,NaN,LA FLORIDA
190389,URBANA,INVASION,SIN DATO,NaN,NUEVE DE ENERO
33816,URBANA,INVASION,SIN DATO,NaN,BRISAS DE COMUNEROS
186963,URBANA,INVASION,SIN DATO,NaN,CINTA BELISARIO
27860,URBANA,INVASION,SIN DATO,NaN,BRISAS DE COMUNEROS
160722,URBANA,INVASION,SIN DATO,NaN,BRISAS DE COMUNEROS
145339,URBANA,INVASION,SIN DATO,NaN,NUEVA ILUSION
15265,URBANA,INVASION,SIN DATO,NaN,LA FLORIDA


#### DERIVAR COLUMNA ZOI (ZONA DE INTERES)

In [83]:
#OTROS CASOS (VIA XXXX, )
mask_via_zona_interes = ((df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("VIA ")) |
                         (df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("VIA AL MAR")) | 
                         (df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("VIA CALI JAMUNDI")) | 
                         (df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("VIA CALI PALMIRA")) |
                         (df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("VIA JAMUNDI")) |
                         (df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("VIA LA SIRENA")) |
                         (df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("VIA LA BUITRERA")) |
                         (df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("KM 18 VIA AL MAR")) |
                         (df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("CERRO TRES CRUCES")) |
                         (df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("CRISTO REY")) |
                         (df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("ZONA AGRICOLA"))                    
                        )
print(mask_via_zona_interes.sum())

df_gen_2010a2018_cali['ZOI_NOMBRE'] = df_gen_2010a2018_cali.BARRIO_NOMBRE[mask_via_zona_interes]

#Limpiar de barrio_nombre los casos detectados
df_gen_2010a2018_cali['BARRIO_NOMBRE'] = df_gen_2010a2018_cali.BARRIO_NOMBRE.replace(to_replace = df_gen_2010a2018_cali.BARRIO_NOMBRE[mask_via_zona_interes], value="ZONA INTERES", regex=True)


541


In [84]:
from IPython.display import display
display(df_gen_2010a2018_cali[['ZONA','BARRIO_NOMBRE','CGTO_NOMBRE', 'VEREDA_NOMBRE', 'INVASION_NOMBRE', 'ZOI_NOMBRE']][mask_via_zona_interes].sample(10))

,ZONA,BARRIO_NOMBRE,CGTO_NOMBRE,VEREDA_NOMBRE,INVASION_NOMBRE,ZOI_NOMBRE
128795,RURAL,ZONA INTERES,SIN DATO,NaN,NaN,VIA LA SIRENA
216823,RURAL,ZONA INTERES,SIN DATO,NaN,NaN,KM 18 VIA AL MAR
68514,RURAL,ZONA INTERES,SIN DATO,NaN,NaN,VIA LA BUITRERA
118996,RURAL,ZONA INTERES,SIN DATO,NaN,NaN,VIA JAMUNDI
149661,URBANA,ZONA INTERES,SIN DATO,NaN,NaN,KM 18 VIA AL MAR
93631,URBANA,ZONA INTERES,SIN DATO,NaN,NaN,CRISTO REY
168649,RURAL,ZONA INTERES,SIN DATO,NaN,NaN,KM 18 VIA AL MAR
118199,RURAL,ZONA INTERES,SIN DATO,NaN,NaN,VIA LA BUITRERA
90187,RURAL,ZONA INTERES,SIN DATO,NaN,NaN,CRISTO REY
153217,RURAL,ZONA INTERES,SIN DATO,NaN,NaN,CRISTO REY


#### DERIVAR COLUMNA METROPOLITANA_NOMBRE

In [85]:
#AREA METROPOLITANA
mask_area_metropolitana = ((df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("PALMIRA")) | 
                           (df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("YUMBO")) | 
                           (df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("JAMUNDI")) |
                           (df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("CANDELARIA")) |
                           (df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("LA CUMBRE")) |
                           (df_gen_2010a2018_cali['BARRIO_NOMBRE'].str.contains("VIJES")) 
                           )                    
print(mask_area_metropolitana.sum())

df_gen_2010a2018_cali['METROPOLITANA_NOMBRE'] = df_gen_2010a2018_cali.BARRIO_NOMBRE[mask_area_metropolitana]

#Limpiar de barrio_nombre los casos detectados
df_gen_2010a2018_cali['BARRIO_NOMBRE'] = df_gen_2010a2018_cali.BARRIO_NOMBRE.replace(to_replace = df_gen_2010a2018_cali.BARRIO_NOMBRE[mask_area_metropolitana], value="AREA METROPOLITANA", regex=True)


14


In [86]:
from IPython.display import display
display(df_gen_2010a2018_cali[['ZONA','BARRIO_NOMBRE','CGTO_NOMBRE', 'VEREDA_NOMBRE', 'INVASION_NOMBRE', 'ZOI_NOMBRE', 'METROPOLITANA_NOMBRE']][mask_area_metropolitana].sample(10))

,ZONA,BARRIO_NOMBRE,CGTO_NOMBRE,VEREDA_NOMBRE,INVASION_NOMBRE,ZOI_NOMBRE,METROPOLITANA_NOMBRE
125952,URBANA,AREA METROPOLITANA,SIN DATO,NaN,NaN,NaN,JAMUNDI
134537,URBANA,AREA METROPOLITANA,SIN DATO,NaN,NaN,NaN,JAMUNDI
56436,URBANA,AREA METROPOLITANA,SIN DATO,NaN,NaN,NaN,CANDELARIA
39482,URBANA,AREA METROPOLITANA,SIN DATO,NaN,NaN,NaN,YUMBO
45068,URBANA,AREA METROPOLITANA,SIN DATO,NaN,NaN,NaN,YUMBO
38606,URBANA,AREA METROPOLITANA,SIN DATO,NaN,NaN,NaN,YUMBO
46214,RURAL,AREA METROPOLITANA,SIN DATO,NaN,NaN,NaN,CANDELARIA
134535,URBANA,AREA METROPOLITANA,SIN DATO,NaN,NaN,NaN,JAMUNDI
134536,URBANA,AREA METROPOLITANA,SIN DATO,NaN,NaN,NaN,JAMUNDI
49624,RURAL,AREA METROPOLITANA,SIN DATO,NaN,NaN,NaN,VIJES


In [87]:
df_barrios_fase4 = gerate_df_from_valuecounts(df_gen_2010a2018_cali['BARRIO_NOMBRE'].value_counts(), ['barrio', 'conteo'])

El dataframe tiene tamaño de:  (466, 2)


#### ** 2.2 LIMPIEZA DE COLUMNA [BARRIO_NOMBRE] CON DICCIONARIO **
>**Nota**
El diccionario es archivo JSON

In [88]:
import json

def json_read(filename):
    with open(filename, encoding='utf-8') as f_in:
    #with open(filename) as f_in:
        return(json.load(f_in))


In [89]:
dict_barriors = json_read('./datos_oficiales/diccionario_barrios.json')
dict_barriors2=dict_barriors
#print(dict_barriors2)

In [90]:
##--------------
## FUNCION PARA LIMPIAR CAMPO BARRIOS

def getKeysByValuefromDict3(valueToFind):
    import json
    
    with open('./datos_oficiales/diccionario_barrios.json', encoding='utf-8') as f_in:
        dictOfElements = json.load(f_in)
    
    #print(dictOfElements)

    listOfKeys = list()
    listOfItems = dictOfElements.items()
    word = valueToFind
    for item  in listOfItems:
        my_item_list = item[1]
        #print (my_item_list)
        for p in my_item_list: 
            #print (p)
            if (p == valueToFind):
                #print('ok')
                #listOfKeys.append(item[0])
                listOfKeys = item[0]
                #print(listOfKeys)
                word = item[0]
                #print(word)
                
    return word

#Forma interesante
#lKey = [key for key, value in lDictionary.iteritems() if value == lValue][0]

# def translate(in_value, translation_dict):
#     out_value = in_value
#     for k, v in translation_dict.iteritems():
#         if out_value in v:
#             out_value = k
#             break
#     return out_value


In [91]:
df_gen_2010a2018_cali['BARRIO_OFICIAL']=''
#df_gen_2010a2018.barrio_nombre_oficial = df_gen_2010a2018.barrio_nombre_oficial.str.encode('utf-8')

df_gen_2010a2018_cali['BARRIO_OFICIAL'] = df_gen_2010a2018_cali.BARRIO_NOMBRE.apply(getKeysByValuefromDict3)


In [92]:
print('--- Reporte de limpieza ---')
print('Valores unicos de BARRIO_TEMP   : ', len(df_gen_2010a2018_cali.BARRIO_TEMP.unique()))
print('Valores unicos de BARRIO_NOMBRE : ', len(df_gen_2010a2018_cali.BARRIO_NOMBRE.unique()))
print('Valores unicos de BARRIO_OFICIAL: ', len(df_gen_2010a2018_cali.BARRIO_OFICIAL.unique()))

--- Reporte de limpieza ---
Valores unicos de BARRIO_TEMP   :  526
Valores unicos de BARRIO_NOMBRE :  466
Valores unicos de BARRIO_OFICIAL:  430


In [93]:
df_barrios_fase5 = gerate_df_from_valuecounts(df_gen_2010a2018_cali['BARRIO_OFICIAL'].value_counts(), ['barrio', 'conteo'])

El dataframe tiene tamaño de:  (430, 2)


In [94]:
import datetime
now = datetime.datetime.now()

ruta = './my_logs/'
nombre_base = 'log_depuracion_barrio'
fecha = str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'-'+str(now.hour).zfill(2)+str(now.minute).zfill(2)
nombre_archivo = ruta + nombre_base + fecha +'.xlsx'

writer = pd.ExcelWriter(nombre_archivo)
df_barrios_fase1.to_excel(writer, 'fase_1')
df_barrios_fase2.to_excel(writer, 'fase_2')
df_barrios_fase3.to_excel(writer, 'fase_3')
df_barrios_fase4.to_excel(writer, 'fase_4_prev_dict')
df_barrios_fase5.to_excel(writer, 'fase_5_post_dict')
writer.save()

In [95]:
from IPython.display import display
mask_prueba = (df_gen_2010a2018_cali.BARRIO_OFICIAL== 'LA PAZ')
print(len(mask_prueba))
display(df_gen_2010a2018_cali['COMUNA_DEV'][mask_prueba].value_counts())

217449


02    66
Name: COMUNA_DEV, dtype: int64

In [96]:
df_gen_2010a2018_cali[df_gen_2010a2018_cali.BARRIO_NOMBRE.str.contains('SEGUROS P.')].sample(5)

,FECHA,DEPARTAMENTO_,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE_SITIO,ARMA_EMPLEADA,MOVIL_AGRESOR,...,BARRIO_TEMP,ESTACION_COD_DEV,COMUNA_DEV,BARRIO_NOMBRE,CGTO_NOMBRE,VEREDA_NOMBRE,INVASION_NOMBRE,ZOI_NOMBRE,METROPOLITANA_NOMBRE,BARRIO_OFICIAL
64564,2013-09-12 00:00:00,VALLE,CALI (CT),JUEVES,15:00:00,CAÑAVERALEJO - SEGUROS P. E19,URBANA,VIAS PUBLICAS,SIN EMPLEO DE ARMAS,A PIE,...,CAÑAVERALEJO - SEGUROS P.,E-19,19,CAÑAVERALEJO - SEGUROS P.,SIN DATO,NaN,NaN,NaN,NaN,CAÑAVERALEJO - SEGUROS PATRIA
118658,08/15/2018 12:00:00 AM,VALLE,CALI (CT),MIÉRCOLES,12/31/1899 07:30:00 AM,CAÑAVERALEJO - SEGUROS P. E19,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,...,CAÑAVERALEJO - SEGUROS P.,E-19,19,CAÑAVERALEJO - SEGUROS P.,SIN DATO,NaN,NaN,NaN,NaN,CAÑAVERALEJO - SEGUROS PATRIA
137414,2012-01-01 00:00:00,VALLE,CALI (CT),DOMINGO,18:40:00,CAÑAVERALEJO - SEGUROS P. E19,URBANA,VIAS PUBLICAS,CONTUNDENTES,A PIE,...,CAÑAVERALEJO - SEGUROS P.,E-19,19,CAÑAVERALEJO - SEGUROS P.,SIN DATO,NaN,NaN,NaN,NaN,CAÑAVERALEJO - SEGUROS PATRIA
12265,2018-08-27,VALLE,CALI (CT),LUNES,14:00:00,CAÑAVERALEJO - SEGUROS P. E19,URBANA,VIAS PUBLICAS,SIN EMPLEO DE ARMAS,A PIE,...,CAÑAVERALEJO - SEGUROS P.,E-19,19,CAÑAVERALEJO - SEGUROS P.,SIN DATO,NaN,NaN,NaN,NaN,CAÑAVERALEJO - SEGUROS PATRIA
116931,2018-04-07 00:00:00,VALLE,CALI (CT),MIÉRCOLES,12/31/1899 05:20:00 PM,CAÑAVERALEJO - SEGUROS P. E19,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,CONDUCTOR MOTOCICLETA,...,CAÑAVERALEJO - SEGUROS P.,E-19,19,CAÑAVERALEJO - SEGUROS P.,SIN DATO,NaN,NaN,NaN,NaN,CAÑAVERALEJO - SEGUROS PATRIA


#### ** 2.3 Mapeo columna ['BARRIO_OFICIAL'] con bases de datos oficial **

>**Nota**
Se deriva las columnas ['BARRIO_COD'], ['COMUNA'],['AREA'] por medio de un merge dataframes  


In [108]:
df_gen_2010a2018_cali_temp = df_gen_2010a2018_cali.copy()

In [99]:
file_name = './datos_oficiales/cali_barrios_oficial.xlsx'
xls_file = pd.read_excel(file_name)
print(xls_file.shape)
xls_file.head()

(338, 8)


,id_barrio,barrio,cod_barrio,barrio_2,barrio_nombre_oficial,comuna_oficial,estrato_moda,area_km2
0,101,Terron Colorado,101,TERRON COLORADO,TERRON COLORADO,1,2,1.386998
1,102,Vista Hermosa,102,VISTA HERMOSA,VISTA HERMOSA,1,1,1.148898
2,196,Sector Patio Bonito,196,SECTOR PATIO BONITO,SECTOR PATIO BONITO,1,1,0.198371
3,199,Aguacatal,199,AGUACATAL,AGUACATAL,1,1,1.107991
4,201,Santa Rita,201,SANTA RITA,SANTA RITA,2,6,0.181149


In [100]:
# df_barrios_oficiales
df_barrios_oficiales = pd.DataFrame()
df_barrios_oficiales[['BARRIO_OFICIAL', 'COD_BARRIO', 'COMUNA_OFICIAL','AREA_km2']] = xls_file[['barrio_nombre_oficial', 'cod_barrio', 'comuna_oficial','area_km2']]
df_barrios_oficiales.sample(5)

,BARRIO_OFICIAL,COD_BARRIO,COMUNA_OFICIAL,AREA_km2
319,PIZAMOS I,2101,21,0.179983
93,ALFONSO LOPEZ P. 1A. ETAPA,701,7,0.586055
134,EL DORADO,1001,10,0.155725
220,LAUREANO GOMEZ,1503,15,0.223410
49,MANZANARES,405,4,0.275037


In [101]:
df_barrios_oficiales.COMUNA_OFICIAL = df_barrios_oficiales.COMUNA_OFICIAL.astype(str)
df_barrios_oficiales.COMUNA_OFICIAL = df_barrios_oficiales.COMUNA_OFICIAL.str.zfill(2)

df_barrios_oficiales.COD_BARRIO = df_barrios_oficiales.COD_BARRIO.astype(str)
df_barrios_oficiales.COD_BARRIO = df_barrios_oficiales.COD_BARRIO.str.zfill(4)

#df_barrio_area_km2.barrio_nombre_oficial = df_barrio_area_km2.barrio_nombre_oficial.str.encode('utf-8')
from IPython.display import display
display(df_barrios_oficiales.sample(5))


,BARRIO_OFICIAL,COD_BARRIO,COMUNA_OFICIAL,AREA_km2
69,LOS GUAYACANES,0503,05,0.450897
56,LA ISLA,0413,04,0.147751
276,LA CASCADA,1902,19,0.211618
215,LOS NARANJOS I,1496,14,0.082976
44,ACUEDUCTO SAN ANTONIO,0316,03,0.241054


In [102]:
#df_barrios_oficiales.BARRIO_OFICIAL.unique()

In [103]:
#df_barrios_oficiales.BARRIO_OFICIAL[df_barrios_oficiales.BARRIO_OFICIAL.str.contains('SEGUROS')]

In [109]:
df_gen_2010a2018_cali_temp.sample(5)

,FECHA,DEPARTAMENTO_,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE_SITIO,ARMA_EMPLEADA,MOVIL_AGRESOR,...,BARRIO_TEMP,ESTACION_COD_DEV,COMUNA_DEV,BARRIO_NOMBRE,CGTO_NOMBRE,VEREDA_NOMBRE,INVASION_NOMBRE,ZOI_NOMBRE,METROPOLITANA_NOMBRE,BARRIO_OFICIAL
185303,2011-02-21 00:00:00,VALLE,CALI (CT),LUNES,18:00:00,SUCRE E9,URBANA,DENTRO DE LA VIVIENDA,CONTUNDENTES,A PIE,...,SUCRE,E-9,09,SUCRE,SIN DATO,NaN,NaN,NaN,NaN,SUCRE
167884,2017-04-08 00:00:00,VALLE,CALI (CT),SÁBADO,06:30:00,SANTA ELENA E10,URBANA,PLAZAS DE MERCADO,CONTUNDENTES,A PIE,...,SANTA ELENA,E-10,10,SANTA ELENA,SIN DATO,NaN,NaN,NaN,NaN,SANTA ELENA
21416,2018-04-23 00:00:00,VALLE,CALI (CT),LUNES,05:30:00,AGUACATAL E1,URBANA,VIAS PUBLICAS,NO REPORTADO,A PIE,...,AGUACATAL,E-1,01,AGUACATAL,SIN DATO,NaN,NaN,NaN,NaN,AGUACATAL
112470,03/22/2018 12:00:00 AM,VALLE,CALI (CT),JUEVES,12/31/1899 07:30:00 AM,LILI E17,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,CONDUCTOR MOTOCICLETA,...,LILI,E-17,17,LILI,SIN DATO,NaN,NaN,NaN,NaN,LILI
42690,2010-12-09 00:00:00,VALLE,CALI (CT),JUEVES,13:00:00,LA FLORESTA E8,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,ARMA DE FUEGO,CONDUCTOR MOTOCICLETA,...,LA FLORESTA,E-8,08,LA FLORESTA,SIN DATO,NaN,NaN,NaN,NaN,LA FLORESTA


In [110]:
#Se hace el merge o la fusión
df_gen_2010a2018_cali_temp = pd.merge(df_gen_2010a2018_cali_temp, df_barrios_oficiales, how='left')

In [111]:
df_gen_2010a2018_cali_temp.sample(5)

,FECHA,DEPARTAMENTO_,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE_SITIO,ARMA_EMPLEADA,MOVIL_AGRESOR,...,BARRIO_NOMBRE,CGTO_NOMBRE,VEREDA_NOMBRE,INVASION_NOMBRE,ZOI_NOMBRE,METROPOLITANA_NOMBRE,BARRIO_OFICIAL,COD_BARRIO,COMUNA_OFICIAL,AREA_km2
129957,2010-07-01 00:00:00,VALLE,CALI (CT),JUEVES,00:00:00,PASEO DE LOS ALMENDROS E5,URBANA,VIAS PUBLICAS,CONTUNDENTES,A PIE,...,PASEO DE LOS ALMENDROS,SIN DATO,NaN,NaN,NaN,NaN,PASEO DE LOS ALMENDROS,0596,05,0.176183
117518,07/19/2018 12:00:00 AM,VALLE,CALI (CT),JUEVES,12/31/1899 11:00:00 AM,LOS ROBLES E13,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,ARMA DE FUEGO,PASAJERO MOTOCICLETA,...,LOS ROBLES,SIN DATO,NaN,NaN,NaN,NaN,LOS ROBLES,1311,13,0.148971
7513,2016-05-26,VALLE,CALI (CT),JUEVES,00:00:00,CGTO LA LEONERA E1,URBANA,VIAS PUBLICAS,SIN EMPLEO DE ARMAS,A PIE,...,CORREGIMIENTO,LA LEONERA,NaN,NaN,NaN,NaN,CORREGIMIENTO,NaN,NaN,NaN
55643,2012-09-11 00:00:00,VALLE,CALI (CT),MARTES,08:30:00,CALIMA E4,URBANA,CAJERO AUTOMATICO,SIN EMPLEO DE ARMAS,A PIE,...,CALIMA,SIN DATO,NaN,NaN,NaN,NaN,CALIMA,0421,04,0.441970
195805,2014-02-02 00:00:00,VALLE,CALI (CT),DOMINGO,20:30:00,CGTO LA LEONERA E1,URBANA,CASAS DE HABITACION,SIN EMPLEO DE ARMAS,A PIE,...,CORREGIMIENTO,LA LEONERA,NaN,NaN,NaN,NaN,CORREGIMIENTO,NaN,NaN,NaN


In [112]:
df_gen_2010a2018_cali_temp.COD_BARRIO = df_gen_2010a2018_cali_temp.COD_BARRIO.fillna('SIN DATO')
df_gen_2010a2018_cali_temp.COMUNA_OFICIAL = df_gen_2010a2018_cali_temp.COMUNA_OFICIAL.fillna('SIN DATO')

In [113]:
import numpy as np
df_gen_2010a2018_cali_temp['VALIDADOR'] = np.where((df_gen_2010a2018_cali_temp.COMUNA_DEV == df_gen_2010a2018_cali_temp.COMUNA_OFICIAL), True, False)

In [114]:
df_gen_2010a2018_cali_temp['BARRIO_OFICIAL'][(df_gen_2010a2018_cali_temp['VALIDADOR']==False)].value_counts()

CORREGIMIENTO                 6280
INVASION                       993
ZONA INTERES                   541
VILLA SAN MARCOS               457
CAÑAVERAL - BELLA SUIZA         99
URB. SANTA ELENA                97
VEREDA                          88
LA BASE                         73
SEC LOS MANGOS                  60
VILLACOLOMBIA                   59
PAR. MONACO                     58
COMFENALCO                      52
PANCE                           48
QUINTAS DEL SOL                 46
BELALCAZAR                      45
LAS DELICIAS                    41
CAMILO TORRES                   40
LAS PALMAS                      39
SECTOR LA NAVE                  39
REMANSOS DE COMFANDI            38
LA LUISA                        37
VILLANUEVA                      35
SOL DE ORIENTE                  34
DESEPAZ INVICALI                33
SANTA BARBARA                   30
GUALANDAY                       30
CIUDAD MODELO                   28
QUINTAS DE SALOMIA              27
PANAMERICANO        

In [115]:
df_gen_2010a2018_cali_temp[['BARRIO_OFICIAL', 'COMUNA_DEV', 'COMUNA_OFICIAL']][(df_gen_2010a2018_cali_temp['BARRIO_OFICIAL']=='SEC LOS MANGOS')]

,BARRIO_OFICIAL,COMUNA_DEV,COMUNA_OFICIAL
715,SEC LOS MANGOS,19,SIN DATO
1247,SEC LOS MANGOS,19,SIN DATO
6113,SEC LOS MANGOS,19,SIN DATO
23380,SEC LOS MANGOS,19,SIN DATO
23510,SEC LOS MANGOS,19,SIN DATO
23928,SEC LOS MANGOS,19,SIN DATO
24661,SEC LOS MANGOS,19,SIN DATO
24662,SEC LOS MANGOS,19,SIN DATO
26415,SEC LOS MANGOS,19,SIN DATO
26416,SEC LOS MANGOS,19,SIN DATO


#### ** 2.4 Guardar los casos .csv **

>**Nota**


In [116]:
import datetime
now = datetime.datetime.now()

ruta = './df_2010u2017_pg_cali/'
nombre_base = 'df_gen_2010a2019_cali_'
fecha = str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'-'+str(now.hour).zfill(2)+str(now.minute).zfill(2)
nombre_archivo = ruta + nombre_base + fecha +'.csv'

df_gen_2010a2018_cali_temp.reset_index(drop=True)
df_gen_2010a2018_cali_temp.to_csv(nombre_archivo, index = False)

In [117]:
df_gen_2010a2018_cali_temp.ARMA_EMPLEADA.unique()

array(['CONTUNDENTES', 'NO REPORTADO', 'ARMA DE FUEGO', 'ARMA BLANCA',
       'SIN EMPLEO DE ARMAS', '-', 'ARMA BLANCA / CORTOPUNZANTE',
       'ESCOPOLAMINA', 'CORTOPUNZANTES', 'LICOR ADULTERADO',
       'CINTAS/CINTURON', 'CORTANTES', 'ESPOSAS', 'CUERDA/SOGA/CADENA',
       'PUNZANTES', 'GASOLINA', 'CASA BOMBA', 'COMBUSTIBLE', 'LIQUIDOS',
       'GRANADA DE MANO', 'PERRO', 'LLAVE MAESTRA', 'JERINGA',
       'POLVORA(FUEGOS PIROTECNICOS)', 'VENENO', 'MOTO',
       'SUSTANCIAS TOXICAS', 'ARTEFACTO EXPLOSIVO/CARGA DINAMITA',
       'MEDICAMENTOS', 'AGUA CALIENTE', 'QUIMICOS', 'VEHICULO', 'GASES',
       'CINTAS', 'PAPA EXPLOSIVA', 'ACIDO', 'ARTEFACTO INCENDIARIO',
       'ALIMENTOS VENCIDOS', 'ALUCINOGENOS', 'CUCHILLA', 'BOLSA PLASTICA',
       'DIRECTA'], dtype=object)

In [118]:
##Se generará uno compacto para reducir consumo de computo

columnas = ['FECHA_FORMATO', 'AÑO', 'MES', 'SEMANA_NUMERO','DIA_NUMERO','DIA','FECHAS_ESPECIALES', 
            'HORA_FORMATO_24h', 'HORA_24x', 'ZONA', 'CLASE_SITIO', 'ARMA_EMPLEADA', 'MOVIL_AGRESOR', 'MOVIL_VICTIMA',
            'SEXO', 'EDAD_FORMATO', 'EDAD_5Q_80mas', 'EDAD_NNAJ','ESTADO_CIVIL', 'PAIS_NACIMIENTO', 'CLASE_EMPLEADO',
            'PROFESION', 'ESCOLARIDAD', 'CODIGO_DANE', 'CANTIDAD', 'CONFLICTIVIDAD', 'BARRIO_NOMBRE', 'ESTACION_COD_DEV',
            'COMUNA_DEV',  'CGTO_NOMBRE', 'VEREDA_NOMBRE', 'INVASION_NOMBRE', 'ZOI_NOMBRE','METROPOLITANA_NOMBRE', 'BARRIO_OFICIAL', 'COD_BARRIO',
            'COMUNA_OFICIAL', 'AREA_km2']

df_gen_2010a2018_cali_compact = df_gen_2010a2018_cali_temp[columnas].copy()


In [119]:
import datetime
now = datetime.datetime.now()

ruta = './df_2010u2017_pg_cali/'
nombre_base = 'df_gen_2010a2019_cali_compact_'
fecha = str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'-'+str(now.hour).zfill(2)+str(now.minute).zfill(2)
nombre_archivo = ruta + nombre_base + fecha +'.csv'

df_gen_2010a2018_cali_compact.reset_index(drop=True)
df_gen_2010a2018_cali_compact.to_csv(nombre_archivo, index = False)

In [120]:
import datetime
from pivottablejs import pivot_ui
now = datetime.datetime.now()

ruta = './pivot_tables/'
nombre_base = 'OPC-Pivot_table_gen_cali_'
fecha = str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'-'+str(now.hour).zfill(2)+str(now.minute).zfill(2)
nombre_archivo_pvt = ruta + nombre_base + fecha +'.html'

pivot_ui(df_gen_2010a2018_cali_compact, outfile_path=nombre_archivo_pvt)


____
### **----- ANULAR 1 inicio ----**

In [50]:
# Se identificaran los casos asociados a la ciudad de CALI
#Inicialmente buscamos dentro de la columna municipio cuales es contiene la palabra CALI
#Para detectar las formas como se refieren a este municipio

#Hacemos una mascara para filtar los datos con CALI
casos_con_cali_index_amz = df_2010u2018_amz_raw['MUNICIPIO'].str.contains('CALI')
df_casos_con_cali_amz = (df_2010u2018_amz_raw[casos_con_cali_index_amz])
df_casos_con_cali_counts_amz =df_casos_con_cali_amz['MUNICIPIO'].value_counts()
print(df_casos_con_cali_counts_amz)

# #Hacemos una mascara para filtar los datos con CALI
casos_con_cali_index_ds = df_2010u2018_ds_raw['MUNICIPIO'].str.contains('CALI')
df_casos_con_cali_ds = (df_2010u2018_ds_raw[casos_con_cali_index_ds])
df_casos_con_cali_counts_ds =df_casos_con_cali_ds['MUNICIPIO'].value_counts()
print(df_casos_con_cali_counts_ds)

#Hacemos una mascara para filtar los datos con CALI
casos_con_cali_index_ho = df_2010u2018_ho_raw['MUNICIPIO'].str.contains('CALI')
df_casos_con_cali_ho = (df_2010u2018_ho_raw[casos_con_cali_index_ho])
df_casos_con_cali_counts_ho =df_casos_con_cali_ho['MUNICIPIO'].value_counts()
print(df_casos_con_cali_counts_ho)

# #Hacemos una mascara para filtar los datos con CALI
casos_con_cali_index_hp = df_2010u2018_hp_raw['MUNICIPIO'].str.contains('CALI')
df_casos_con_cali_hp = (df_2010u2018_hp_raw[casos_con_cali_index_hp])
df_casos_con_cali_counts_hp =df_casos_con_cali_hp['MUNICIPIO'].value_counts()
print(df_casos_con_cali_counts_hp)

# #Hacemos una mascara para filtar los datos con CALI
casos_con_cali_index_lp = df_2010u2018_lp_raw['MUNICIPIO'].str.contains('CALI')
df_casos_con_cali_lp = (df_2010u2018_lp_raw[casos_con_cali_index_lp])
df_casos_con_cali_counts_lp =df_casos_con_cali_lp['MUNICIPIO'].value_counts()
print(df_casos_con_cali_counts_lp)

# #Hacemos una mascara para filtar los datos con CALI
casos_con_cali_index_vi = df_2010u2018_vi_raw['MUNICIPIO'].str.contains('CALI')
df_casos_con_cali_vi = (df_2010u2018_vi_raw[casos_con_cali_index_vi])
df_casos_con_cali_counts_vi =df_casos_con_cali_vi['MUNICIPIO'].value_counts()
print(df_casos_con_cali_counts_vi)

CALI (CT)      13697
CALIMA            87
SAN CALIXTO       74
CALIFORNIA        18
Name: MUNICIPIO, dtype: int64
CALI (CT)      8834
CALIMA           94
SAN CALIXTO      12
CALIFORNIA        9
Name: MUNICIPIO, dtype: int64
CALI (CT)      13947
CALIMA            91
SAN CALIXTO       41
Name: MUNICIPIO, dtype: int64
CALI (CT)      91053
CALIMA           188
CALIFORNIA        13
SAN CALIXTO        5
Name: MUNICIPIO, dtype: int64
CALI (CT)      55066
CALIMA           317
CALIFORNIA        46
SAN CALIXTO       34
Name: MUNICIPIO, dtype: int64
CALI (CT)      34852
CALIMA           168
CALIFORNIA        28
SAN CALIXTO        4
Name: MUNICIPIO, dtype: int64


In [55]:
cali_index_amz = (df_2010u2018_amz_raw['MUNICIPIO'] == 'CALI (CT)')
df_2010u2018_amz_raw_cali = df_2010u2018_amz_raw[cali_index_amz]

cali_index_ds = (df_2010u2018_ds_raw['MUNICIPIO'] == 'CALI (CT)')
df_2010u2018_ds_raw_cali = df_2010u2018_ds_raw[cali_index_ds]

cali_index_ho = (df_2010u2018_ho_raw['MUNICIPIO'] == 'CALI (CT)')
df_2010u2018_ho_raw_cali = df_2010u2018_ho_raw[cali_index_ho]

cali_index_hp = (df_2010u2018_hp_raw['MUNICIPIO'] == 'CALI (CT)')
df_2010u2018_hp_raw_cali = df_2010u2018_hp_raw[cali_index_hp]

cali_index_lp = (df_2010u2018_lp_raw['MUNICIPIO'] == 'CALI (CT)')
df_2010u2018_lp_raw_cali = df_2010u2018_lp_raw[cali_index_lp]

cali_index_vi = (df_2010u2018_vi_raw['MUNICIPIO'] == 'CALI (CT)')
df_2010u2018_vi_raw_cali = df_2010u2018_vi_raw[cali_index_vi]

In [56]:
print('Los registros del df asociados a CALI en 2010 a 2109 es: ' + str(df_2010u2018_amz_raw_cali.shape))
print('Los registros del df asociados a CALI en 2010 a 2109 es: ' + str(df_2010u2018_ds_raw_cali.shape))
print('Los registros del df asociados a CALI en 2010 a 2109 es: ' + str(df_2010u2018_ho_raw_cali.shape))
print('Los registros del df asociados a CALI en 2010 a 2109 es: ' + str(df_2010u2018_hp_raw_cali.shape))
print('Los registros del df asociados a CALI en 2010 a 2109 es: ' + str(df_2010u2018_lp_raw_cali.shape))
print('Los registros del df asociados a CALI en 2010 a 2109 es: ' + str(df_2010u2018_vi_raw_cali.shape))

Los registros del df asociados a CALI en 2010 a 2109 es: (13697, 20)
Los registros del df asociados a CALI en 2010 a 2109 es: (8834, 21)
Los registros del df asociados a CALI en 2010 a 2109 es: (13947, 20)
Los registros del df asociados a CALI en 2010 a 2109 es: (91053, 20)
Los registros del df asociados a CALI en 2010 a 2109 es: (55066, 20)
Los registros del df asociados a CALI en 2010 a 2109 es: (34852, 20)


In [6]:
import datetime
now = datetime.datetime.now()
nombre_base_ho = './df_2010u2017_raw_cali/df_2010u2018_ho_raw_cali_'
nombre_base_hp = './df_2010u2017_raw_cali/df_2010u2018_hp_raw_cali_'
nombre_base_lp = './df_2010u2017_raw_cali/df_2010u2018_lp_raw_cali_'
nombre_base_vi = './df_2010u2017_raw_cali/df_2010u2018_vi_raw_cali_'

nombre_archivo_ho = nombre_base_ho + str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'.csv'
nombre_archivo_hp = nombre_base_hp + str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'.csv'
nombre_archivo_lp = nombre_base_lp + str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'.csv'
nombre_archivo_vi = nombre_base_vi + str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'.csv'

df_2010u2018_ho_raw_cali.reset_index(drop=True)
df_2010u2018_ho_raw_cali.to_csv(nombre_archivo_ho, index = False)

df_2010u2018_hp_raw_cali.reset_index(drop=True)
df_2010u2018_hp_raw_cali.to_csv(nombre_archivo_hp, index = False)

df_2010u2018_lp_raw_cali.reset_index(drop=True)
df_2010u2018_lp_raw_cali.to_csv(nombre_archivo_lp, index = False)

df_2010u2018_vi_raw_cali.reset_index(drop=True)
df_2010u2018_vi_raw_cali.to_csv(nombre_archivo_vi, index = False)

print(df_2010u2018_ho_raw_cali.shape)
print(df_2010u2018_hp_raw_cali.shape)
print(df_2010u2018_lp_raw_cali.shape)
print(df_2010u2018_vi_raw_cali.shape)

(13775, 20)
(88693, 20)
(53195, 20)
(32731, 20)


In [6]:
import datetime
now = datetime.datetime.now()
nombre_base_ho = './df_2010u2017_raw_cali/df_2010u2018_ho_raw_cali_'
nombre_base_hp = './df_2010u2017_raw_cali/df_2010u2018_hp_raw_cali_'
nombre_base_lp = './df_2010u2017_raw_cali/df_2010u2018_lp_raw_cali_'
nombre_base_vi = './df_2010u2017_raw_cali/df_2010u2018_vi_raw_cali_'

nombre_archivo_ho = nombre_base_ho + str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'.csv'
nombre_archivo_hp = nombre_base_hp + str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'.csv'
nombre_archivo_lp = nombre_base_lp + str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'.csv'
nombre_archivo_vi = nombre_base_vi + str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'.csv'

df_2010u2018_ho_raw_cali.reset_index(drop=True)
df_2010u2018_ho_raw_cali.to_csv(nombre_archivo_ho, index = False)

df_2010u2018_hp_raw_cali.reset_index(drop=True)
df_2010u2018_hp_raw_cali.to_csv(nombre_archivo_hp, index = False)

df_2010u2018_lp_raw_cali.reset_index(drop=True)
df_2010u2018_lp_raw_cali.to_csv(nombre_archivo_lp, index = False)

df_2010u2018_vi_raw_cali.reset_index(drop=True)
df_2010u2018_vi_raw_cali.to_csv(nombre_archivo_vi, index = False)

print(df_2010u2018_ho_raw_cali.shape)
print(df_2010u2018_hp_raw_cali.shape)
print(df_2010u2018_lp_raw_cali.shape)
print(df_2010u2018_vi_raw_cali.shape)

(13775, 20)
(88693, 20)
(53195, 20)
(32731, 20)


### ----- ANULAR 1 final ----

In [92]:
df_gen_2010a2018_temp_validador = df_gen_2010a2018_temp[df_gen_2010a2018_temp['validador1']==False]
df_gen_2010a2018_temp_validador.shape

print(df_gen_2010a2018_temp_validador.barrio_nombre.unique())
print(df_gen_2010a2018_temp_validador.barrio_nombre_oficial.unique())


import datetime
now = datetime.datetime.now()

ruta = './df_2010u2017_pg_cali/'
nombre_base = 'barrios_no_coincide_sus_comunas'
fecha = str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'-'+str(now.hour).zfill(2)+str(now.minute).zfill(2)
nombre_archivo = ruta + nombre_base + fecha +'.csv'

df_gen_2010a2018_temp_validador.reset_index(drop=True)
df_gen_2010a2018_temp_validador.to_csv(nombre_archivo, index = False)

['LA PRADERA' 'CHAMPAGNAT' 'PANCE' '' 'LAS VERANERAS' 'LOS CEREZOS'
 'VILLA BLANCA' 'CAÑAVERALEJO' 'BAJOS CIUDAD CORDOBA' 'SECTOR LA NAVE'
 'MULTICENTRO' 'LAS PALMAS I' 'CAMILO TORRES' 'PAZ' 'SEMINARIO'
 'COMFENALCO' 'PAR. MONACO' 'LAS PALMAS' 'VILLA SAN MARCOS' 'GUALANDAY'
 'REALENGO' 'SOL DE ORIENTE' 'VILLA LUZ' 'LA RIVERA' 'LA LUISA' 'LA BASE'
 'VILLACOLOMBIA' 'VILLA MERCEDES I' 'SEC LOS MANGOS' 'QUINTAS DEL SOL'
 'LAS DELICIAS' 'PALERMO' 'LARES DE COMFENALCO' 'SANTA BARBARA' 'CAMELIAS'
 'LA MARIA' 'OASIS DE COMFANDI' 'QUINTAS DE SALOMIA' 'LA PORTADA' 'PACARA'
 'MOJICA II' 'REMANSOS DE COMFANDI' 'BRISAS DEL GUAVITO' 'VILLACATALINA'
 '-' 'MANANTIAL' 'TEJARES DE SALOMIA' 'SECTOR CHORRITOS' 'BELEN'
 'INVICALI' 'BELLA  SUIZA' 'PLAZAS VERDES' 'CALI BELLA' 'EL PILAR'
 'URIBE URIBE' 'EL SAMAN' 'VILLA ISABEL' 'SECTOR LA TORRE' 'SANTA CLARA'
 'ALFONSO 3' 'CIUDAD MODELO' 'VILLA DEL MAR' 'TERCER MILENIO' 'EL ESTERO'
 'URBANIZACION DESEPAZ' 'ALFONSO 2' 'VIILA DEL PARQUE'
 'VILLAS DE VERACRUZ' '

In [93]:
print(df_gen_2010a2018_temp_validador.barrio_nombre_oficial.value_counts())

                              4839
-                              686
PANCE                          598
VILLA SAN MARCOS               421
CHAMPAGÑAT                     399
MOJICA II                      318
REMANSOS DE COMFANDI           289
LA BASE                        286
LA RIVERA                      274
VILLA BLANCA                   234
CIUDAD MODELO                  214
SANTA BARBARA                  214
COMFENALCO                     201
QUINTAS DEL SOL                189
TERCER MILENIO                 181
LA PAZ                         176
VILLACOLOMBIA                  160
URIBE URIBE                    151
LAS DELICIAS                   147
BAJOS CIUDAD CORDOBA           145
LAS PALMAS                     131
SOL DE ORIENTE                 130
QUINTAS DE SALOMIA             128
DESEPAZ INVICALI               117
SECTOR LA NAVE                 102
CAÑAVERALEJO                    93
ALFONSO LOPEZ P. 3A. ETAPA      93
CAMILO TORRES                   92
VILLAS DE VERACRUZ  